# Similar Neighborhoods
### Import packages

In [1]:
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation

# from bs4 import BeautifulSoup
#  http://beautiful-soup-4.readthedocs.io/en/latest/    # for more advanced web scraping  

import lxml
import html5lib
import csv

import json # library to handle JSON files

import requests # library to handle requests

import time  # used to record time to run functions
from collections import Counter

import geocoder # import geocoder
import geopy
from geopy.geocoders import Nominatim

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

from simpledbf import Dbf5  

print('Libraries imported.')

PyTables is not installed. No support for HDF output.
Libraries imported.


In [2]:
import sys

sys.path.insert(1, '../scripts/')
import Functions_similiar_neighboorhoods as func 

# insert at 1, 0 is the script path (or '' in REPL)
sys.path.insert(1, '../credentials/')
import config

# Import neighborhood names data

## Toronto
### start with Toronto import

In [3]:
# url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
# df_to_neighborhoods = pd.read_html(url, attrs={"class": "wikitable"})[0]   # 0 is for the 1st table in this particular page
# df_to_neighborhoods.head()

In [4]:
url = 'https://www.zipcodesonline.com/2020/06/postal-code-of-toronto-in-2020.html'
df_to_neighborhoods = pd.read_html(url,header=0)[1]   # 0 is for the 1st table in this particular page
df_to_neighborhoods.head(5)

,SL. NO.,NEIGHBOURHOOD,POSTAL CODES,DISTRICT
0,NaN,NaN,NaN,NaN
1,1.0,Adelaide,M5H,Downtown Toronto
2,2.0,Agincourt North,M1V,Scarborough
3,3.0,Agincourt,M1S,Scarborough
4,4.0,Albion Gardens,M9V,Etobicoke


## New York
### NYC data import

In [5]:
dbf_ny_neighborhoods= Dbf5('../datafiles/nyu_neighborhoods.dbf')
df_ny_neighborhoods = dbf_ny_neighborhoods.to_dataframe()
df_ny_neighborhoods.head(3)

,Name,Stacked,AnnoLine1,AnnoLine2,AnnoLine3,AnnoAngle,Borough
0,Wakefield,1,Wakefield,NaN,NaN,0.0,Bronx
1,Co-op City,2,Co-op,City,NaN,0.0,Bronx
2,Eastchester,1,Eastchester,NaN,NaN,0.0,Bronx


# Clean data

## New York
### clean NYC data

Remove and rename columns

In [6]:
df_ny_neighborhoods = df_ny_neighborhoods.drop(['Stacked','AnnoLine1','AnnoLine2','AnnoLine3','AnnoAngle'], axis=1) 
df_ny_neighborhoods = df_ny_neighborhoods.rename(columns={"Name":"Neighborhood"})
df_ny_neighborhoods.head(5)

,Neighborhood,Borough
0,Wakefield,Bronx
1,Co-op City,Bronx
2,Eastchester,Bronx
3,Fieldston,Bronx
4,Riverdale,Bronx


First duplicates and NaN values are explored in the Toronto data

In [7]:
df_ny_neighborhoods.isna().sum()

Neighborhood    0
Borough         0
dtype: int64

Good we do not have any NaN values

Let's remove the Postal Codes column since it's not needed.

In [8]:
count = 0
for i,d in enumerate(df_ny_neighborhoods['Neighborhood'].duplicated()):
    if d == True:
        count= count + 1
        print(i,d)
        print(df_ny_neighborhoods['Neighborhood'].iloc[i])
print(str(count) + " Neighborhoods with duplicates") 

180 True
Murray Hill
220 True
Sunnyside
235 True
Bay Terrace
244 True
Chelsea
4 Neighborhoods with duplicates


In [9]:
count = 0
for i,d in enumerate(df_ny_neighborhoods['Borough'].duplicated()):
    if d == True:
        count= count + 1
        # print(i,d)
        # print(df_ny_neighborhoods['Borough'].iloc[i])
print(str(count) + " Borough with duplicates") 

301 Borough with duplicates


In [10]:
count = 0
for i,d in enumerate(df_ny_neighborhoods.duplicated()):
    if d == True:
        count= count + 1
        print(i,d)
        print(df_ny_neighborhoods.iloc[i])
print(str(count) + " rows with duplicates") 

0 rows with duplicates


Let's examine the names

In [11]:
pd.set_option('display.max_rows', None)
# df_ny_neighborhoods.head(400)
# pd.set_option('display.max_rows', 50)

All seems good

## Toronto
### clean the Toronto dataframe

Remove and rename columns

In [12]:
df_to_neighborhoods.drop(columns=["SL. NO."], inplace=True )

df_to_neighborhoods.dropna(axis='index', inplace=True )

df_to_neighborhoods.rename(columns={"NEIGHBOURHOOD" : "Neighborhood", "POSTAL CODES" : "Postal Code", "DISTRICT" : "Borough"}, inplace=True)

df_to_neighborhoods.head()

,Neighborhood,Postal Code,Borough
1,Adelaide,M5H,Downtown Toronto
2,Agincourt North,M1V,Scarborough
3,Agincourt,M1S,Scarborough
4,Albion Gardens,M9V,Etobicoke
5,Alderwood,M8W,Etobicoke


First duplicates and NaN values are explored in the Toronto data

In [13]:
# Which rows has 'Postal Code' duplicates
count = 0
for i,d in enumerate(df_to_neighborhoods['Postal Code'].duplicated()):
    if d == True:
        count = count + 1
        # print(i,d)
        # print(df_to_neighborhoods['Postal Code'].iloc[i])
print(str(count) + " Postal Codes with duplicates")

105 Postal Codes with duplicates


In [14]:
df_to_neighborhoods.isna().sum()

Neighborhood    0
Postal Code     0
Borough         0
dtype: int64

Good we do not have any NaN values

Let's remove the Postal Codes column since it's not needed.

In [15]:
df_to_neighborhoods = df_to_neighborhoods.drop(['Postal Code'], axis=1) 
df_to_neighborhoods.head()

,Neighborhood,Borough
1,Adelaide,Downtown Toronto
2,Agincourt North,Scarborough
3,Agincourt,Scarborough
4,Albion Gardens,Etobicoke
5,Alderwood,Etobicoke


In [16]:
count = 0
for i,d in enumerate(df_to_neighborhoods['Neighborhood'].duplicated()):
    if d == True:
        count= count + 1
        # print(i,d)
        # print(df_to_neighborhoods['Neighborhood'].iloc[i])
print(str(count) + " Neighborhoods with duplicates") 

4 Neighborhoods with duplicates


In [17]:
count = 0
for i,d in enumerate(df_to_neighborhoods['Borough'].duplicated()):
    if d == True:
        count= count + 1
        # print(i,d)
        # print(df_to_neighborhoods['Borough'].iloc[i])
print(str(count) + " Boroughs with duplicates")

194 Boroughs with duplicates


In [18]:
# Duplicated rows
for i,d in enumerate(df_to_neighborhoods.duplicated()):
    if d == True:
        print(i,d)
        print(df_to_neighborhoods.iloc[i])

42 True
Neighborhood     Downsview
Borough         North York
Name: 43, dtype: object
93 True
Neighborhood      Lawrence Park
Borough         Central Toronto
Name: 94, dtype: object
157 True
Neighborhood      St. James Town
Borough         Downtown Toronto
Name: 158, dtype: object


In [19]:
df_to_neighborhoods[df_to_neighborhoods['Neighborhood'].str.match('Downsview')]

,Neighborhood,Borough
41,Downsview,North York
42,Downsview North,North York
43,Downsview,North York


In [20]:
df_to_neighborhoods[df_to_neighborhoods['Neighborhood'].str.match('Lawrence Park')]

,Neighborhood,Borough
93,Lawrence Park,Central Toronto
94,Lawrence Park,Central Toronto


In [21]:
df_to_neighborhoods[df_to_neighborhoods['Neighborhood'].str.match('St. James Town')]

,Neighborhood,Borough
157,St. James Town,Downtown Toronto
158,St. James Town,Downtown Toronto


There are several duplicates of 'Postal Code's (removed), 'Borough's. There are four duplicates of 'Neighborhood's. There are three duplicate rows, and these three will be removed.

In [22]:
df_to_neighborhoods.drop(index=[41,93,157], inplace=True)

Let's examin the naming:

In [23]:
df_to_neighborhoods.head(210)

,Neighborhood,Borough
1,Adelaide,Downtown Toronto
2,Agincourt North,Scarborough
3,Agincourt,Scarborough
4,Albion Gardens,Etobicoke
5,Alderwood,Etobicoke
6,Bathurst Manor,North York
7,Bathurst Quay,Downtown Toronto
8,Bayview Village,North York
9,Beaumond Heights,Etobicoke
10,"Bedford Park, ,",North York


I saw that index 9 had a weird naming with "," in it. Let's check for more of these

In [24]:
df_to_neighborhoods['Borough'][df_to_neighborhoods['Borough'].str.lower().str.contains(',', regex=False)]

38    North York,
Name: Borough, dtype: object

In [25]:
df_to_neighborhoods.iloc[37]

Neighborhood      Don Mills
Borough         North York,
Name: 38, dtype: object

In [26]:
df_to_neighborhoods['Borough'] = df_to_neighborhoods['Borough'].str.replace(',', '')
# df_to_neighborhoods['Borough'][df_to_neighborhoods['Borough'].str.lower().str.contains('North York', regex=False)]
df_to_neighborhoods.iloc[37]

Neighborhood     Don Mills
Borough         North York
Name: 38, dtype: object

In [27]:
df_to_neighborhoods['Neighborhood'][df_to_neighborhoods['Neighborhood'].str.lower().str.contains(',', regex=False)]

10           Bedford Park, ,
69     Humber Bay, Etobicoke
201        Woodbine Gardens,
Name: Neighborhood, dtype: object

In [28]:
df_to_neighborhoods.iloc[9]

Neighborhood    Bedford Park, ,
Borough              North York
Name: 10, dtype: object

For index 9 we can just remove ", ,"

In [29]:
df_to_neighborhoods['Neighborhood'] = df_to_neighborhoods['Neighborhood'].str.replace(', ,', '')
df_to_neighborhoods.iloc[9]

Neighborhood    Bedford Park
Borough           North York
Name: 10, dtype: object

In [30]:
df_to_neighborhoods.iloc[68]

Neighborhood    Humber Summit
Borough            North York
Name: 70, dtype: object

For index 68 we can just remove "Etobicoke" since it's included int the column borough

In [31]:
df_to_neighborhoods['Neighborhood'] = df_to_neighborhoods['Neighborhood'].str.replace(', Etobicoke', '')
df_to_neighborhoods.iloc[68]

Neighborhood    Humber Summit
Borough            North York
Name: 70, dtype: object

In [32]:
df_to_neighborhoods.iloc[200]

Neighborhood    York Mills
Borough         North York
Name: 204, dtype: object

In [33]:
df_to_neighborhoods['Neighborhood'] = df_to_neighborhoods['Neighborhood'].str.replace(',', '')
df_to_neighborhoods.iloc[200]

Neighborhood    York Mills
Borough         North York
Name: 204, dtype: object

Let's check the number of rows in our two data sets

In [34]:
print('TO ',df_to_neighborhoods.shape)
print('NYC ',df_ny_neighborhoods.shape)

TO  (202, 2)
NYC  (306, 2)


## Add location data of the neighborhoods

Geocoding API geocoder is used

Split the data into three partitions to make it easier / increase the chance of a successful import from the geo API

In [35]:
df_ny_neighborhoods_1 = df_ny_neighborhoods[0:int(df_ny_neighborhoods.shape[0]/3)]
df_ny_neighborhoods_2 = df_ny_neighborhoods[int(df_ny_neighborhoods.shape[0]/3):2*int(df_ny_neighborhoods.shape[0]/3)]
df_ny_neighborhoods_3 = df_ny_neighborhoods[2*int(df_ny_neighborhoods.shape[0]/3):df_ny_neighborhoods.shape[0]]

In [36]:
lat_ny_list_1,long_ny_list_1 = func.get_coordinates_neighborhoods(df_ny_neighborhoods_1,'New York City, USA', 'Neighborhood' )


Progress : Get all Coordinates
Percent: [#####-----] 46.0% Manhattan Terrace   count = 50: TimedOut. Number 53
Percent: [########--] 78.0% Prospect Lefferts Gardens   count = 50: TimedOut. Number 88
Percent: [#########-] 86.0% South Side   count = 50: TimedOut. Number 97
Percent: [#########-] 90.0% Done!


In [37]:
# for i, n in enumerate(df_to_neighborhoods['Neighborhood']):
#     print(str(i)+" " +str(n)+ " " + str(df_to_neighborhoods['Borough'].iloc[i]))

In [38]:
# df_to_neighborhoods[['Neighborhood', 'Borough']]

In [39]:
# locator = Nominatim(user_agent="myGeocoder")
# print(1)
# for i, n in enumerate(df_to_neighborhoods['Neighborhood']):
#     d=df_to_neighborhoods['Borough'].iloc[i]
#     u = '{}, {} '.format(n,d )+ 'Toronto, Ontario, Canada'
#     location = locator.geocode(u)
# # location = locator.geocode('The Annex, Central Toronto, Toronto, Ontario, Canada')
#     print(u)
#     print(location)

In [40]:
# df_ny_neighborhoods_2[~df_ny_neighborhoods_2['Neighborhood'].str.contains('Kew Gardens Hills', case = False)] # 'Kew Gardens Hills' created some kind of problem 

In [41]:
time.sleep(60) # make sure API does not get overwhelmed and dies on me :/
lat_ny_list_2,long_ny_list_2 = func.get_coordinates_neighborhoods(df_ny_neighborhoods_2,'New York City, USA', 'Neighborhood' )


Progress : Get all Coordinates
Percent: [#####-----] 48.0% Kew Gardens Hills   count = 50: TimedOut. Number 55
Percent: [######----] 64.0% Bay Terrace   count = 50: TimedOut. Number 73
Percent: [########--] 75.0% Lindenwood   count = 50: TimedOut. Number 85
Percent: [#########-] 88.0% West Brighton   count = 50: TimedOut. Number 99
Percent: [#########-] 90.0% Done!


In [42]:
time.sleep(60) # make sure API does not get overwhelmed and dies on me :/
lat_ny_list_3,long_ny_list_3 = func.get_coordinates_neighborhoods(df_ny_neighborhoods_3,'New York City, USA', 'Neighborhood' )


Progress : Get all Coordinates
Percent: [##--------] 15.0% Wingate   count = 50: TimedOut. Number 18
Percent: [##--------] 17.0% Park Hill   count = 50: TimedOut. Number 20
Percent: [#####-----] 52.0% Jamaica Hills   count = 50: TimedOut. Number 59
Percent: [#######---] 67.0% Vinegar Hill   count = 50: TimedOut. Number 76
Percent: [#######---] 71.0% Manor Heights   count = 50: TimedOut. Number 80
Percent: [#######---] 72.0% Sandy Ground   count = 50: TimedOut. Number 82
Percent: [#########-] 90.0% Done!


In [43]:
######## count 50 
# Progress : Get all Coordinates
# Percent: [----------] 0.0% Williamsbridge   count = 100: TimedOut
# Percent: [#---------] 10.0% Soundview   count = 100: TimedOut
# Percent: [#---------] 10.0% North Riverdale   count = 100: TimedOut
# Percent: [#---------] 10.0% Olinville   count = 100: TimedOut
# Percent: [#---------] 10.0% Pelham Gardens   count = 100: TimedOut
# Percent: [#---------] 10.0% Edenwald   count = 100: TimedOut
# Percent: [##--------] 20.0% Manhattan Terrace   count = 100: TimedOut
# Percent: [###-------] 30.0% Prospect Lefferts Gardens   count = 100: TimedOut
# Percent: [###-------] 30.0% Central Harlem   count = 100: TimedOut
# Percent: [#####-----] 50.0% Kew Gardens Hills   count = 100: TimedOut
# Percent: [######----] 60.0% Holliswood   count = 100: TimedOut
# Percent: [######----] 60.0% Lindenwood   count = 100: TimedOut
# Percent: [######----] 60.0% North Corona   count = 100: TimedOut
# Percent: [#######---] 70.0% Rugby   count = 100: TimedOut
# Percent: [########--] 80.0% Butler Manor   count = 100: TimedOut
# Percent: [########--] 80.0% Arden Heights   count = 100: TimedOut
# Percent: [########--] 80.0% Richmond Town   count = 100: TimedOut
# Percent: [########--] 80.0% New Lots   count = 100: TimedOut
# Percent: [#########-] 90.0% Jamaica Hills   count = 100: TimedOut
# Percent: [#########-] 90.0% Astoria Heights   count = 100: TimedOut
# Percent: [#########-] 90.0% Claremont Village   count = 100: TimedOut
# Percent: [#########-] 90.0% Mount Eden   count = 100: TimedOut
# Percent: [#########-] 90.0% Fulton Ferry   count = 100: TimedOut
# Percent: [#########-] 90.0% Weeksville   count = 100: TimedOut
# Percent: [#########-] 90.0% Sandy Ground   count = 100: TimedOut
# Percent: [#########-] 90.0% Egbertville   count = 100: TimedOut
# Percent: [#########-] 90.0% Homecrest   count = 100: TimedOut
# Percent: [##########] 100% Done...
# Percent: [##########] 100% Done...
# Percent: [##########] 100% Done...
# Percent: [##########] 100% Done...
# Percent: [##########] 100% Done...
# Percent: [##########] 100% Done...
# Bronxdale   count = 100: TimedOut
# Percent: [##########] 100% Done...
# Allerton   count = 100: TimedOut
# Percent: [##########] 100% Done...
# Percent: [##########] 100% Done...
# Erasmus   count = 100: TimedOut
# Percent: [##########] 100% Done...
# Percent: [##########] 100% Done...
# Percent: [##########] 100% Done...
# Percent: [##########] 100% Done...
# Queensbridge   count = 100: TimedOut
# Percent: [##########] 100% Done...
# Percent: [##########] 100% Done...
# Done!

######## count 100
# Progress : Get all Coordinates 
# Percent: [#---------] 10.0% North Riverdale   count = 100: TimedOut
# Percent: [#---------] 10.0% Pelham Gardens   count = 100: TimedOut
# Percent: [#---------] 10.0% Edenwald   count = 100: TimedOut
# Percent: [##--------] 20.0% Manhattan Terrace   count = 100: TimedOut
# Percent: [###-------] 30.0% Prospect Lefferts Gardens   count = 100: TimedOut
# Percent: [###-------] 30.0% Central Harlem   count = 100: TimedOut
# Percent: [#####-----] 50.0% 
# canceled


######## count 500
# Progress : Get all Coordinates
# Percent: [----------] 3.0% Williamsbridge   count = 100: TimedOut
# Percent: [#---------] 8.0% Soundview   count = 100: TimedOut
# Percent: [#---------] 11.0% North Riverdale   count = 100: TimedOut
# Percent: [#---------] 13.0% Olinville   count = 100: TimedOut
# Percent: [#---------] 13.0% Pelham Gardens   count = 100: TimedOut
# Percent: [#---------] 14.000000000000002% Edenwald   count = 100: TimedOut
# Percent: [##--------] 16.0% Manhattan Terrace   count = 100: TimedOut
# Percent: [###-------] 28.000000000000004% Prospect Lefferts Gardens   count = 100: TimedOut
# Percent: [###-------] 33.0% Central Harlem   count = 100: TimedOut
# Percent: [#####-----] 49.0% 

In [44]:
lat_ny_list = pd.DataFrame(data=np.concatenate([lat_ny_list_1, lat_ny_list_2, lat_ny_list_3]) , columns=["Latitude"])
long_ny_list = pd.DataFrame(data=np.concatenate([long_ny_list_1, long_ny_list_2, long_ny_list_3]) , columns=["Longitude"])
df_ny_neighborhoods = pd.concat([df_ny_neighborhoods_1, df_ny_neighborhoods_2, df_ny_neighborhoods_3], sort=False)

In [45]:
df_ny_neighborhoods["Latitude"]=lat_ny_list
df_ny_neighborhoods["Longitude"]=long_ny_list
df_ny_neighborhoods.head()

,Neighborhood,Borough,Latitude,Longitude
0,Wakefield,Bronx,40.9062,-73.8555
1,Co-op City,Bronx,40.8739,-73.8294
2,Eastchester,Bronx,40.8884,-73.8282
3,Fieldston,Bronx,40.8944,-73.9033
4,Riverdale,Bronx,40.9006,-73.9064


In [46]:
# d = {'Neighborhood': ["Wakefield", "Co-op City", "Eastchester", "Fieldston", "Riverdale"], 'Borough': ["Bronx", "Bronx","Bronx","Bronx","Bronx"]}
# df_ny_neighborhoods_test = pd.DataFrame(data=d)

In [47]:
# for i, neigh in enumerate(df_ny_neighborhoods_test["Neighborhood"]):
#         # initialize your variable to None
#         lat_coords =None
#         lng_coords =None
#         locator = Nominatim(user_agent="myGeocoder")

#         # loop until you get the coordinates
#         while(lng_coords is None):
#             lat_coords =None
#             lng_coords =None

#             location = locator.geocode('{}, New York City, USA'.format(neigh))
#             try:
#                 lat_coords = location.latitude
#                 lng_coords = location.longitude
#                 print("neigh ",str(neigh)," lat ",str(lat_coords)," long ",str(lng_coords))

#             except:
#                 lat_coords =None
#                 lng_coords =None
#                 print("neigh ",str(neigh), " blaaa")

In [48]:
# df_ny_neighborhoods=pd.DataFrame(df_ny_neighborhoods)

In [49]:
# df_ny_neighborhoods['Neighborhood']

In [50]:
df_ny_neighborhoods.iloc[df_ny_neighborhoods.index[df_ny_neighborhoods['Neighborhood'].str.contains('North Riverdale')][0]]

Neighborhood    North Riverdale
Borough                   Bronx
Latitude                40.9046
Longitude              -73.9137
Name: 36, dtype: object

In [51]:

# n=True
# count=0
# locator = Nominatim(user_agent="myGeocoder")
# while n ==True:
#     locator = Nominatim(user_agent="myGeocoder")
#     try:
        
#         location = locator.geocode('North Riverdale, New York City, USA')
#         lat_coords = location.latitude
#         print(lat_coords)
#         n=False
#     except:
#         count=count+1
#         print(count)

In [52]:
# lat_ny_list,long_ny_list = func.get_coordinates_neighborhoods(df_ny_neighborhoods_test,'New York City', 'Neighborhood' )

In [53]:
# df_ny_neighborhoods_test["Latitude"]=lat_ny_list
# df_ny_neighborhoods_test["Longitude"]=long_ny_list
# df_ny_neighborhoods_test.head()

In [54]:
# df_ny_neighborhoods.tail(34)

In [55]:
lat_to_list,long_to_list = func.get_coordinates_neighborhoods(df_to_neighborhoods,'Toronto, Ontario, Canada', 'Neighborhood' )


Progress : Get all Coordinates
Percent: [----------] 3.0% Beaumond Heights   count = 50: TimedOut. Number 8
Percent: [----------] 4.0% Berczy Park   count = 50: TimedOut. Number 10
Percent: [#---------] 7.000000000000001% Business reply mail Processing Centre   count = 50: TimedOut. Number 15
Percent: [#---------] 8.0% Caledonia Fairbanks   count = 50: TimedOut. Number 17
Percent: [#---------] 8.0% Canada Post Gateway Processing Centre   count = 50: TimedOut. Number 18
Percent: [#---------] 9.0% Central Bay Street   count = 50: TimedOut. Number 20
Percent: [###-------] 26.0% Grange Park   count = 50: TimedOut. Number 56
Percent: [###-------] 28.000000000000004% Harbourfront West   count = 50: TimedOut. Number 60
Percent: [###-------] 31.0% Humber Bay Shores   count = 50: TimedOut. Number 66
Percent: [###-------] 34.0% India Bazaar   count = 50: TimedOut. Number 73
Percent: [####------] 35.0% Island airport   count = 50: TimedOut. Number 75
Percent: [####------] 36.0% Keelsdale and Sil

In [56]:
df_to_neighborhoods["Latitude"]=lat_to_list
df_to_neighborhoods["Longitude"]=long_to_list
df_to_neighborhoods.head()

,Neighborhood,Borough,Latitude,Longitude
1,Adelaide,Downtown Toronto,43.6503,-79.3805
2,Agincourt North,Scarborough,43.808,-79.2664
3,Agincourt,Scarborough,43.7854,-79.2785
4,Albion Gardens,Etobicoke,43.7437,-79.6067
5,Alderwood,Etobicoke,43.6017,-79.5452


In [57]:
print('TO ',df_to_neighborhoods.shape)
print('NYC ',df_ny_neighborhoods.shape)

TO  (202, 4)
NYC  (306, 4)


Check number of nan values in coordinates:

In [58]:
df_to_neighborhoods.isna().sum()

Neighborhood     0
Borough          0
Latitude        21
Longitude       21
dtype: int64

In [59]:
df_ny_neighborhoods.isna().sum()

Neighborhood     0
Borough          0
Latitude        13
Longitude       13
dtype: int64

Remove all rows with no coordinate value

In [60]:
df_ny_neighborhoods.dropna( axis=0, how="any", thresh=None, subset=None, inplace=True)
df_to_neighborhoods.dropna( axis=0, how="any", thresh=None, subset=None, inplace=True)

In [64]:
df_to_neighborhoods.isna().sum()

Neighborhood    0
Borough         0
Latitude        0
Longitude       0
dtype: int64

In [123]:
df_to_neighborhoods.reset_index(drop=False, inplace=True, col_level=0, col_fill='')

# Now adding weather data of the neighborhoods

we use openweathermap api to get last 5 days daily temperature and the most frequent weather to compare. 
   An average of the neighborhoods temperature and the most common weathers would have been preferred.

In [61]:
openweathermap_api_key = config.openweathermap_api_key

### Toronoto

In [130]:
weather_mean_temp_to, weather_most_freq_weather_to, weather_2nd_most_freq_weather_to = func.get_weather_data(df_to_neighborhoods.iloc[[170,171,172,173,174,175,176, 177,178,179, 180]], openweathermap_api_key)

2021-04-26 22:58:39,443 - DEBUG - Starting new HTTPS connection (1): api.openweathermap.org:443
2021-04-26 22:58:39,443 - DEBUG - Starting new HTTPS connection (1): api.openweathermap.org:443
2021-04-26 22:58:39,606 - DEBUG - https://api.openweathermap.org:443 "GET /data/2.5/onecall?lat=43.7001608&lon=-79.5162474&exclude=current,minutely,hourly,alerts&appid=5f46cb0d859d8771c9f6380b82b10d5a HTTP/1.1" 200 4093
2021-04-26 22:58:39,606 - DEBUG - https://api.openweathermap.org:443 "GET /data/2.5/onecall?lat=43.7001608&lon=-79.5162474&exclude=current,minutely,hourly,alerts&appid=5f46cb0d859d8771c9f6380b82b10d5a HTTP/1.1" 200 4093
2021-04-26 22:58:43,617 - DEBUG - Starting new HTTPS connection (1): api.openweathermap.org:443
2021-04-26 22:58:43,617 - DEBUG - Starting new HTTPS connection (1): api.openweathermap.org:443
2021-04-26 22:58:43,783 - DEBUG - https://api.openweathermap.org:443 "GET /data/2.5/onecall?lat=43.7432421&lon=-79.304641&exclude=current,minutely,hourly,alerts&appid=5f46cb0d8

IndexError: index 1 is out of bounds for axis 0 with size 1

In [70]:
# import logging
# logging.basicConfig(level=logging.DEBUG, format='%(asctime)s - %(levelname)s - %(message)s')
# logging.debug('This is a log message.')

2021-04-26 21:34:54,637 - DEBUG - This is a log message.


In [81]:
# import logging
# logger = logging.getLogger()
# logger.setLevel(logging.DEBUG)

# formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')

# fh = logging.FileHandler('log_filename.txt')
# fh.setLevel(logging.DEBUG)
# fh.setFormatter(formatter)
# logger.addHandler(fh)

# ch = logging.StreamHandler()
# ch.setLevel(logging.DEBUG)
# ch.setFormatter(formatter)
# logger.addHandler(ch)

# logger.debug('This is a test log message.')

2021-04-26 21:47:53,227 - DEBUG - This is a test log message.
2021-04-26 21:47:53,227 - DEBUG - This is a test log message.


In [121]:
# df_to_neighborhoods.drop('index', inplace=True, axis=1)

In [122]:
# df_neighborhoods = df_to_neighborhoods.iloc[[172,173,174,175,176, 177,178,179, 180]]
# print(df_neighborhoods)

         Neighborhood          Borough Latitude Longitude
172           Wexford      Scarborough  43.7454  -79.2947
173        Willowdale       North York  43.7615  -79.4109
174    Wilson Heights       North York  43.7405    -79.44
175            Woburn      Scarborough  43.7598  -79.2253
176  Woodbine Gardens        East York  43.7121  -79.3026
177  Woodbine Heights        East York  43.6999  -79.3193
178   York Mills West       North York   43.744  -79.4067
179        York Mills       North York   43.744  -79.4067
180         Yorkville  Central Toronto  43.6714  -79.3902


In [129]:
# df_neighborhoods = df_to_neighborhoods.iloc[[170,171,172,173,174,175,176, 177,178,179, 180]]

# weather_mean_temp=np.array([])
# weather_most_freq_weather=np.array([])
# weather_2nd_most_freq_weather=np.array([])

# # get coordinates for all neighborhoods
# for lat,lon in df_neighborhoods[['Latitude', 'Longitude']].itertuples(index=False):

#     # Get data from api, with coordinates
#     url = "https://api.openweathermap.org/data/2.5/onecall?lat={}&lon={}&exclude={}&appid={}".format(lat, lon, "current,minutely,hourly,alerts", openweathermap_api_key)
#     weather_data = requests.get(url).json()
    
#     # there is a constraint that MAX 60 api-calls per minutes can be made, thus we make lower number of calls per minute by waiting
#     time.sleep(4)
    
#     # get weather data for last 5 days and get mean ans most freq
#     weather_last_5=[]
#     for i in range(0,len(weather_data['daily'])):
#         # print(weather_data['daily'][i]['temp']['day'])
#         # clean and add data in dataframe
#         weather_last_5.append([ round( weather_data['daily'][i]['temp']['day'] -273.15, 2),  weather_data['daily'][i]['weather'][0]['main'] ])
#     df_weather_last_5=pd.DataFrame(weather_last_5, columns=['temp','weather'])
    
#     # calculate mean temp and most frequent weather, the last 5 days
#     logging.info('All : '+str(df_weather_last_5))
#     mean_temperature = df_weather_last_5['temp'].mean()
#     most_freq_weather = df_weather_last_5['weather'].value_counts()[:1].index[0]
#     logging.info('Before second most frequent. most_freq_weather: '+str(df_weather_last_5['weather'].value_counts()[:1].index[0]) + " lat: "+str(lat)+ " long: " + str(lon))
#     logging.info('Before 2 second most frequent . second_most_freq_weather: '+str(df_weather_last_5['weather'].value_counts()[:2]))
#     try:
#         second_most_freq_weather = df_weather_last_5['weather'].value_counts()[:2].index[1]
#     except:
#         second_most_freq_weather = df_weather_last_5['weather'].value_counts()[:1].index[0]
#     logging.info('After second most frequent: '+str(second_most_freq_weather) + " lat: "+str(lat)+ " long: " + str(lon))
#     # print(second_most_freq_weather)
    
#     weather_mean_temp = np.append(weather_mean_temp, mean_temperature)
#     weather_most_freq_weather = np.append(weather_most_freq_weather, most_freq_weather)
#     weather_2nd_most_freq_weather = np.append(weather_2nd_most_freq_weather, second_most_freq_weather)

# print( weather_mean_temp, weather_most_freq_weather, weather_2nd_most_freq_weather)

2021-04-26 22:56:24,572 - DEBUG - Starting new HTTPS connection (1): api.openweathermap.org:443
2021-04-26 22:56:24,572 - DEBUG - Starting new HTTPS connection (1): api.openweathermap.org:443
2021-04-26 22:56:24,734 - DEBUG - https://api.openweathermap.org:443 "GET /data/2.5/onecall?lat=43.7001608&lon=-79.5162474&exclude=current,minutely,hourly,alerts&appid=5f46cb0d859d8771c9f6380b82b10d5a HTTP/1.1" 200 4093
2021-04-26 22:56:24,734 - DEBUG - https://api.openweathermap.org:443 "GET /data/2.5/onecall?lat=43.7001608&lon=-79.5162474&exclude=current,minutely,hourly,alerts&appid=5f46cb0d859d8771c9f6380b82b10d5a HTTP/1.1" 200 4093
2021-04-26 22:56:28,750 - INFO - All :     temp weather
0   8.48    Rain
1   5.96    Rain
2   9.45    Rain
3   8.66    Rain
4   9.26    Rain
5   6.57    Snow
6   7.83    Rain
7  12.95    Rain
2021-04-26 22:56:28,750 - INFO - All :     temp weather
0   8.48    Rain
1   5.96    Rain
2   9.45    Rain
3   8.66    Rain
4   9.26    Rain
5   6.57    Snow
6   7.83    Rain
7

In [62]:
df_to_neighborhoods["mean_temp_last5days"]=weather_mean_temp_to
df_to_neighborhoods["most_freq_weather_last5days"]=weather_most_freq_weather_to
df_to_neighborhoods["2nd_most_freq_weather_last5days"]=weather_2nd_most_freq_weather_to
df_to_neighborhoods.head()

,Neighborhood,Borough,Latitude,Longitude,mean_temp_last5days,most_freq_weather_last5days,2nd_most_freq_weather_last5days
1,Adelaide,Downtown Toronto,43.6503,-79.3805,10.77375,Rain,Clouds
2,Agincourt North,Scarborough,43.808,-79.2664,12.34375,Rain,Clouds
3,Agincourt,Scarborough,43.7854,-79.2785,12.24750,Rain,Clouds
4,Albion Gardens,Etobicoke,43.7437,-79.6067,14.63125,Rain,Clouds
5,Alderwood,Etobicoke,43.6017,-79.5452,11.11750,Rain,Clouds


In [63]:
# pd.set_option('display.max_columns', None)
# df_to_neighborhoods

### New york

In [ ]:
time.sleep(360) # increase chance of succesful API call

In [34]:
weather_mean_temp_ny, weather_most_freq_weather_ny, weather_2nd_most_freq_weather_ny = func.get_weather_data(df_ny_neighborhoods, openweathermap_api_key)

In [35]:
df_ny_neighborhoods["mean_temp_last5days"]=weather_mean_temp_ny
df_ny_neighborhoods["most_freq_weather_last5days"]=weather_most_freq_weather_ny
df_ny_neighborhoods["2nd_most_freq_weather_last5days"]=weather_2nd_most_freq_weather_ny
df_ny_neighborhoods.head()

,Neighborhood,Borough,Latitude,Longitude,mean_temp_last5days,most_freq_weather_last5days,2nd_most_freq_weather_last5days
0,Wakefield,Bronx,40.9062,-73.8555,-0.36375,Snow,Clear
1,Co-op City,Bronx,40.8739,-73.8294,-0.00375,Snow,Clear
2,Eastchester,Bronx,40.9611,-73.8065,-0.45250,Snow,Clear
3,Fieldston,Bronx,40.8944,-73.9033,-0.38250,Clear,Snow
4,Riverdale,Bronx,40.9006,-73.9064,-0.51750,Clear,Snow


# Now add crime index

### Toronoto

In [36]:
df_to_crime_all=pd.read_csv('../datafiles/Robbery_2014_to_2019.csv')
df_to_crime_all.head(3)

,X,Y,Index_,event_unique_id,occurrencedate,reporteddate,premisetype,ucr_code,ucr_ext,offence,reportedyear,reportedmonth,reportedday,reporteddayofyear,reporteddayofweek,reportedhour,occurrenceyear,occurrencemonth,occurrenceday,occurrencedayofyear,occurrencedayofweek,occurrencehour,MCI,Division,Hood_ID,Neighbourhood,Long,Lat,ObjectId
0,-8.850277e+06,5.429886e+06,1,GO-20141289469,2014/01/06 01:08:00+00,2014/01/06 01:57:00+00,Other,1610,200,Robbery - Mugging,2014,January,6,6,Monday,1,2014.0,January,6.0,6.0,Monday,1,Robbery,D31,27,York University Heights (27),-79.503388,43.769791,1
1,-8.827688e+06,5.435919e+06,4,GO-20141331950,2014/01/12 20:58:00+00,2014/01/12 20:58:00+00,Other,1610,100,Robbery With Weapon,2014,January,12,12,Sunday,20,2014.0,January,12.0,12.0,Sunday,20,Robbery,D42,130,Milliken (130),-79.300468,43.808918,2
2,-8.827688e+06,5.435919e+06,5,GO-20141331950,2014/01/12 20:58:00+00,2014/01/12 20:58:00+00,Other,1610,100,Robbery With Weapon,2014,January,12,12,Sunday,20,2014.0,January,12.0,12.0,Sunday,20,Robbery,D42,130,Milliken (130),-79.300468,43.808918,3


In [ ]:
Clean data

In [37]:
df_to_crime_2019 = df_to_crime_all[ df_to_crime_all['reportedyear']==2019 ]
print(df_to_crime_2019.shape)
df_to_crime_2019.reset_index(inplace=True)
df_to_crime_2019.head(3)

(3417, 29)


,index,X,Y,Index_,event_unique_id,occurrencedate,reporteddate,premisetype,ucr_code,ucr_ext,offence,reportedyear,reportedmonth,reportedday,reporteddayofyear,reporteddayofweek,reportedhour,occurrenceyear,occurrencemonth,occurrenceday,occurrencedayofyear,occurrencedayofweek,occurrencehour,MCI,Division,Hood_ID,Neighbourhood,Long,Lat,ObjectId
0,14002,-8.847359e+06,5.417543e+06,132094,GO-20191084412,2019/06/11 17:30:00+00,2019/06/12 15:00:00+00,Outside,1610,200,Robbery - Mugging,2019,June,12,163,Wednesday,15,2019.0,June,11.0,162.0,Tuesday,17,Robbery,D12,112,Beechborough-Greenbrook (112),-79.477181,43.689671,14003
1,14012,-8.844723e+06,5.418360e+06,132139,GO-2019224056,2019/02/04 20:40:00+00,2019/02/04 20:53:00+00,Commercial,1610,210,Robbery - Business,2019,February,4,35,Monday,20,2019.0,February,4.0,35.0,Monday,20,Robbery,D13,108,Briar Hill-Belgravia (108),-79.453499,43.694981,14013
2,14016,-8.829785e+06,5.414963e+06,132158,GO-2019283112,2019/02/14 08:45:00+00,2019/02/14 12:09:00+00,Commercial,1610,210,Robbery - Business,2019,February,14,45,Thursday,12,2019.0,February,14.0,45.0,Thursday,8,Robbery,D55,64,Woodbine Corridor (64),-79.319305,43.672909,14017


Compute if crimes occured nearby

In [38]:
crime_closeby_to=np.array([])

# get coordinates for all neighborhoods
for lat,lon in df_to_neighborhoods[['Latitude', 'Longitude']].itertuples(index=False):

    # numpy arrays for coordinates to do matrix calculations
    lat_array=df_to_crime_2019[['Lat']].to_numpy()
    long_array=df_to_crime_2019[[ 'Long']].to_numpy()
    
    lat_dist=np.absolute(lat-lat_array)*111139  # convert distance from Degrees to Meters
    long_dist=np.absolute(lon-long_array)*111139
    distance= np.sqrt( np.power(lat_dist, 2) + np.power(long_dist, 2) ) # pythagoras theorem for distance
    
    df_dist=pd.DataFrame(distance, columns=['distance'])

    df_dist["closeby"] = df_dist['distance'] <= 750    # a crime nearby is within 500 meters
    try:
        number_crimes=df_dist["closeby"].value_counts()[True]
    except:
        # print(df_dist["closeby"].value_counts(), )
        number_crimes=0
    
    crime_closeby_to= np.append(crime_closeby_to, number_crimes)

In [93]:
df_to_neighborhoods["number_crimes_recent_nearby"]=crime_closeby
df_to_neighborhoods.head()

,Neighborhood,Borough,Latitude,Longitude,mean_temp_last5days,most_freq_weather_last5days,2nd_most_freq_weather_last5days,number_crimes_recent_nearby
0,Parkwoods,North York,43.7611,-79.3241,-4.40625,Clouds,Snow,19.0
1,Victoria Village,North York,43.7327,-79.3112,-4.17750,Clouds,Clear,2.0
3,"Lawrence Manor, Lawrence Heights",North York,43.7165,-79.4424,-4.33125,Clouds,Snow,6.0
5,"Islington Avenue, Humber Valley Village",Etobicoke,43.6625,-79.5312,-3.89000,Clouds,Snow,1.0
6,"Malvern, Rouge",Scarborough,43.8092,-79.2217,-4.61125,Clouds,Rain,11.0


### NYC

In [70]:
df_ny_crime_all=pd.read_csv('../datafiles/NYPD_Complaint_Data_Historic.csv')


In [103]:
print(df_ny_crime_all.size)
df_ny_crime_all.head(3)

228252360


,CMPLNT_NUM,CMPLNT_FR_DT,CMPLNT_FR_TM,CMPLNT_TO_DT,CMPLNT_TO_TM,ADDR_PCT_CD,RPT_DT,KY_CD,OFNS_DESC,PD_CD,PD_DESC,CRM_ATPT_CPTD_CD,LAW_CAT_CD,BORO_NM,LOC_OF_OCCUR_DESC,PREM_TYP_DESC,JURIS_DESC,JURISDICTION_CODE,PARKS_NM,HADEVELOPT,HOUSING_PSA,X_COORD_CD,Y_COORD_CD,SUSP_AGE_GROUP,SUSP_RACE,SUSP_SEX,TRANSIT_DISTRICT,Latitude,Longitude,Lat_Lon,PATROL_BORO,STATION_NAME,VIC_AGE_GROUP,VIC_RACE,VIC_SEX
0,325341655,02-11-2015,15:00:00,NaN,NaN,73.0,02/11/2015,359,OFFENSES AGAINST PUBLIC ADMINI,749.0,VIOLATION OF ORDER OF PROTECTI,COMPLETED,MISDEMEANOR,BROOKLYN,INSIDE,RESIDENCE - PUBLIC HOUSING,N.Y. HOUSING POLICE,2.0,NaN,TILDEN,405,1009656.0,181287.0,25-44,BLACK,M,NaN,40.664239,-73.908425,"(40.664239422, -73.908425011)",PATROL BORO BKLYN NORTH,NaN,<18,BLACK,M
1,393816841,03-17-2012,10:30:00,03/17/2012,11:00:00,69.0,03/17/2012,344,ASSAULT 3 & RELATED OFFENSES,114.0,OBSTR BREATH/CIRCUL,COMPLETED,MISDEMEANOR,BROOKLYN,INSIDE,RESIDENCE-HOUSE,N.Y. POLICE DEPT,0.0,NaN,NaN,NaN,1014035.0,174133.0,NaN,NaN,NaN,NaN,40.644590,-73.892672,"(40.644589618, -73.892672426)",PATROL BORO BKLYN SOUTH,NaN,45-64,BLACK,F
2,802896158,10-27-2016,13:48:00,11/03/2016,13:49:00,71.0,11/03/2016,578,HARRASSMENT 2,638.0,"HARASSMENT,SUBD 3,4,5",COMPLETED,VIOLATION,BROOKLYN,INSIDE,PUBLIC SCHOOL,N.Y. POLICE DEPT,0.0,NaN,NaN,NaN,1000222.0,179282.0,<18,BLACK,M,NaN,40.658758,-73.942435,"(40.658758183, -73.942434788)",PATROL BORO BKLYN SOUTH,NaN,18-24,BLACK,M


### clean data

In [105]:
df_ny_crime_all['CMPLNT_FR_DT']=df_ny_crime_all['CMPLNT_FR_DT'].replace({'/':'-'},regex=True)
# df_ny_crime_all['year']= pd.DatetimeIndex(df_ny_crime_all['CMPLNT_FR_DT']).year

# select after 2 / 
df_ny_crime_all.head(3)

,CMPLNT_NUM,CMPLNT_FR_DT,CMPLNT_FR_TM,CMPLNT_TO_DT,CMPLNT_TO_TM,ADDR_PCT_CD,RPT_DT,KY_CD,OFNS_DESC,PD_CD,PD_DESC,CRM_ATPT_CPTD_CD,LAW_CAT_CD,BORO_NM,LOC_OF_OCCUR_DESC,PREM_TYP_DESC,JURIS_DESC,JURISDICTION_CODE,PARKS_NM,HADEVELOPT,HOUSING_PSA,X_COORD_CD,Y_COORD_CD,SUSP_AGE_GROUP,SUSP_RACE,SUSP_SEX,TRANSIT_DISTRICT,Latitude,Longitude,Lat_Lon,PATROL_BORO,STATION_NAME,VIC_AGE_GROUP,VIC_RACE,VIC_SEX
0,325341655,02-11-2015,15:00:00,NaN,NaN,73.0,02/11/2015,359,OFFENSES AGAINST PUBLIC ADMINI,749.0,VIOLATION OF ORDER OF PROTECTI,COMPLETED,MISDEMEANOR,BROOKLYN,INSIDE,RESIDENCE - PUBLIC HOUSING,N.Y. HOUSING POLICE,2.0,NaN,TILDEN,405,1009656.0,181287.0,25-44,BLACK,M,NaN,40.664239,-73.908425,"(40.664239422, -73.908425011)",PATROL BORO BKLYN NORTH,NaN,<18,BLACK,M
1,393816841,03-17-2012,10:30:00,03/17/2012,11:00:00,69.0,03/17/2012,344,ASSAULT 3 & RELATED OFFENSES,114.0,OBSTR BREATH/CIRCUL,COMPLETED,MISDEMEANOR,BROOKLYN,INSIDE,RESIDENCE-HOUSE,N.Y. POLICE DEPT,0.0,NaN,NaN,NaN,1014035.0,174133.0,NaN,NaN,NaN,NaN,40.644590,-73.892672,"(40.644589618, -73.892672426)",PATROL BORO BKLYN SOUTH,NaN,45-64,BLACK,F
2,802896158,10-27-2016,13:48:00,11/03/2016,13:49:00,71.0,11/03/2016,578,HARRASSMENT 2,638.0,"HARASSMENT,SUBD 3,4,5",COMPLETED,VIOLATION,BROOKLYN,INSIDE,PUBLIC SCHOOL,N.Y. POLICE DEPT,0.0,NaN,NaN,NaN,1000222.0,179282.0,<18,BLACK,M,NaN,40.658758,-73.942435,"(40.658758183, -73.942434788)",PATROL BORO BKLYN SOUTH,NaN,18-24,BLACK,M


Compute if crimes occured nearby

In [117]:
# df_ny_crime_all['day'] = df_ny_crime_all['RPT_DT'].dt.day
# df_ny_crime_all['month'] = df_ny_crime_all['RPT_DT'].dt.month
# df_ny_crime_all['year'] = df_ny_crime_all['RPT_DT'].dt.year

split_data_date_ny = df_ny_crime_all["CMPLNT_FR_DT"].str.split("-")
data_date_ny = split_data_date_ny.to_list()
data_date_ny
names_date_ny = ["Day", "Month","Year" ]
new_df_date_ny = pd.DataFrame(data=data_date_ny, columns=names_date_ny)
df_ny_crime_all["Year"]=new_df_date_ny["Year"]
df_ny_crime_all.head(2)

,CMPLNT_NUM,CMPLNT_FR_DT,CMPLNT_FR_TM,CMPLNT_TO_DT,CMPLNT_TO_TM,ADDR_PCT_CD,RPT_DT,KY_CD,OFNS_DESC,PD_CD,PD_DESC,CRM_ATPT_CPTD_CD,LAW_CAT_CD,BORO_NM,LOC_OF_OCCUR_DESC,PREM_TYP_DESC,JURIS_DESC,JURISDICTION_CODE,PARKS_NM,HADEVELOPT,HOUSING_PSA,X_COORD_CD,Y_COORD_CD,SUSP_AGE_GROUP,SUSP_RACE,SUSP_SEX,TRANSIT_DISTRICT,Latitude,Longitude,Lat_Lon,PATROL_BORO,STATION_NAME,VIC_AGE_GROUP,VIC_RACE,VIC_SEX,Year
0,325341655,02-11-2015,15:00:00,NaN,NaN,73.0,02/11/2015,359,OFFENSES AGAINST PUBLIC ADMINI,749.0,VIOLATION OF ORDER OF PROTECTI,COMPLETED,MISDEMEANOR,BROOKLYN,INSIDE,RESIDENCE - PUBLIC HOUSING,N.Y. HOUSING POLICE,2.0,NaN,TILDEN,405,1009656.0,181287.0,25-44,BLACK,M,NaN,40.664239,-73.908425,"(40.664239422, -73.908425011)",PATROL BORO BKLYN NORTH,NaN,<18,BLACK,M,2015
1,393816841,03-17-2012,10:30:00,03/17/2012,11:00:00,69.0,03/17/2012,344,ASSAULT 3 & RELATED OFFENSES,114.0,OBSTR BREATH/CIRCUL,COMPLETED,MISDEMEANOR,BROOKLYN,INSIDE,RESIDENCE-HOUSE,N.Y. POLICE DEPT,0.0,NaN,NaN,NaN,1014035.0,174133.0,NaN,NaN,NaN,NaN,40.644590,-73.892672,"(40.644589618, -73.892672426)",PATROL BORO BKLYN SOUTH,NaN,45-64,BLACK,F,2012
2,802896158,10-27-2016,13:48:00,11/03/2016,13:49:00,71.0,11/03/2016,578,HARRASSMENT 2,638.0,"HARASSMENT,SUBD 3,4,5",COMPLETED,VIOLATION,BROOKLYN,INSIDE,PUBLIC SCHOOL,N.Y. POLICE DEPT,0.0,NaN,NaN,NaN,1000222.0,179282.0,<18,BLACK,M,NaN,40.658758,-73.942435,"(40.658758183, -73.942434788)",PATROL BORO BKLYN SOUTH,NaN,18-24,BLACK,M,2016
3,633812343,11-27-2014,19:00:00,11/27/2014,22:30:00,112.0,11/28/2014,104,RAPE,157.0,RAPE 1,COMPLETED,FELONY,QUEENS,INSIDE,RESIDENCE - APT. HOUSE,N.Y. POLICE DEPT,0.0,NaN,NaN,NaN,1025420.0,202485.0,25-44,WHITE HISPANIC,M,NaN,40.722364,-73.851474,"(40.722363687, -73.851473894)",PATROL BORO QUEENS NORTH,NaN,25-44,WHITE,F,2014
4,300349533,12-11-2013,13:30:00,12/11/2013,14:15:00,24.0,12/12/2013,109,GRAND LARCENY,438.0,"LARCENY,GRAND FROM BUILDING (NON-RESIDENCE) UN...",COMPLETED,FELONY,MANHATTAN,INSIDE,DOCTOR/DENTIST OFFICE,N.Y. POLICE DEPT,0.0,NaN,NaN,NaN,992848.0,228356.0,NaN,NaN,NaN,NaN,40.793465,-73.968950,"(40.793464597, -73.968949638)",PATROL BORO MAN NORTH,NaN,45-64,WHITE,F,2013
5,944253539,08-28-2013,11:00:00,08/28/2013,11:30:00,63.0,08/28/2013,341,PETIT LARCENY,333.0,"LARCENY,PETIT FROM STORE-SHOPL",COMPLETED,MISDEMEANOR,BROOKLYN,INSIDE,DEPARTMENT STORE,N.Y. POLICE DEPT,0.0,NaN,NaN,NaN,1007048.0,167105.0,NaN,NaN,NaN,NaN,40.625320,-73.917874,"(40.625319998, -73.917873518)",PATROL BORO BKLYN SOUTH,NaN,NaN,UNKNOWN,D,2013
6,608679532,12-04-2013,18:30:00,12/04/2013,18:40:00,101.0,12/05/2013,361,OFF. AGNST PUB ORD SENSBLTY &,639.0,AGGRAVATED HARASSMENT 2,COMPLETED,MISDEMEANOR,QUEENS,INSIDE,RESIDENCE - APT. HOUSE,N.Y. POLICE DEPT,0.0,NaN,NaN,NaN,1051824.0,158026.0,NaN,WHITE HISPANIC,F,NaN,40.600172,-73.756663,"(40.600171728, -73.756663063)",PATROL BORO QUEENS SOUTH,NaN,45-64,BLACK,F,2013
7,882178622,09-14-2013,08:40:00,09/14/2013,08:40:00,49.0,09/14/2013,578,HARRASSMENT 2,638.0,"HARASSMENT,SUBD 3,4,5",COMPLETED,VIOLATION,BRONX,FRONT OF,STREET,N.Y. POLICE DEPT,0.0,NaN,NaN,NaN,1022583.0,249362.0,45-64,BLACK,M,NaN,40.851041,-73.861441,"(40.851041106, -73.861441322)",PATROL BORO BRONX,NaN,25-44,WHITE HISPANIC,F,2013
8,240183498,01-31-2013,23:47:00,NaN,NaN,40.0,01/31/2013,352,CRIMINAL TRESPASS,205.0,"TRESPASS 2, CRIMINAL",COMPLETED,MISDEMEANOR,BRONX,INSIDE,RESIDENCE - PUBLIC HOUSING,N.Y. HOUSING POLICE,2.0,NaN,NaN,811,1007373.0,232752.0,NaN,NaN,NaN,NaN,40.805504,-73.916477,"(40.805504347, -73.916477252)",PATROL BORO BRONX,NaN,NaN,UNKNOWN,E,2013
9,895680254,09-17-2016,00:05:00,09/17/2016,00:15:00,60.0,09/17/2016,351,CRIMINAL MISCHIEF & RELATED OF,259.0,"CRIMINAL MISCHIEF,UNCLASSIFIED 4",COMPLETED,MISDEMEANOR,BROOKLYN,FRONT OF,RESIDENCE - APT. HOUSE,N.Y. POLICE DEPT,0.0,NaN,NaN,NaN,984501.0,147851.0,UNKNOWN,BLACK,M,NaN,40.572501,-73.999098,"(40.572500982, -73.999097946)",PATROL BORO BKLYN SOUTH,NaN,25-44,BLACK,F,2016


In [107]:
df_ny_crime_all.dropna(subset=['CMPLNT_FR_DT'],inplace=True)

In [123]:
df_ny_crime_2018=df_ny_crime_all[df_ny_crime_all['Year']=="2018"] # better to split like this instead of using contains #result count=16891632 with .size
# df_test=df_ny_crime_all[df_ny_crime_all['CMPLNT_FR_DT'].str.contains('2018')]                                  #result count=16891596

In [125]:
crime_closeby_ny=np.array([])

# get coordinates for all neighborhoods
for lat,lon in df_ny_neighborhoods[['Latitude', 'Longitude']].itertuples(index=False):

    # numpy arrays for coordinates to do matrix calculations
    lat_array=df_ny_crime_2018[['Latitude']].to_numpy()
    long_array=df_ny_crime_2018[[ 'Longitude']].to_numpy()
    
    lat_dist=np.absolute(lat-lat_array)*111139  # convert distance from Degrees to Meters
    long_dist=np.absolute(lon-long_array)*111139
    distance= np.sqrt( np.power(lat_dist, 2) + np.power(long_dist, 2) ) # pythagoras theorem for distance
    
    df_dist=pd.DataFrame(distance, columns=['distance'])

    df_dist["closeby"] = df_dist['distance'] <= 750    # a crime nearby is within 500 meters
    try:
        number_crimes=df_dist["closeby"].value_counts()[True]
    except:
        # print(df_dist["closeby"].value_counts(), )
        number_crimes=0
    
    crime_closeby_ny= np.append(crime_closeby_ny, number_crimes)

# Housing estimate

### Toronto

In [236]:
start_time = time.time()
wrong_amount_of_element=[]
number_of_rows=16

# AirBnb data file for 2019 #http://insideairbnb.com/get-the-data.html

count=0
airbnb_calend_list=[]
with open('../datafiles/airbnb_listings_sum_toronto.csv', newline='',encoding = 'cp850') as csvfile:    
    csv_reader_data = csv.reader(csvfile)
    for row in csv_reader_data: 
        next_row = []

        if count >=1 and np.shape(row)[0]==1: # if count is not header and has "" around the all elements

            row[0]=row[0].replace("\n","")
            redone_csv_reading = csv.reader(row)
            for new_row in redone_csv_reading:

                if np.shape(new_row)[0]==number_of_rows:
                    next_row = new_row

                else:
                    wrong_amount_of_element.append(np.shape(new_row)[0])

        elif count >=1 and np.shape(row)[0]==number_of_rows:
            next_row = row

        elif count==0:

            headers_row = row

        if count >=1:     
            airbnb_calend_list.append(next_row)

        count = count + 1
        if count == 20000:
            break
# print(airbnb_calend_list)
# print(np.shape(airbnb_calend_list),np.shape(headers_row))
df_airbnb_listing_toronto = pd.DataFrame(data=airbnb_calend_list, columns= headers_row)
# print(df_airbnb_listing_toronto.head())

print("Number of elements in a data row and their count: ",Counter(wrong_amount_of_element))
print("--- %s seconds ---" % (time.time() - start_time))

Number of elements in a data row and their count:  Counter({2: 62, 15: 58, 1: 6})
--- 0.283064603805542 seconds ---


In [95]:
# df_airbnb_listing_toronto.head()

Clean dataframe 

In [237]:
# change currency from canadian dollar to Euro
df_airbnb_listing_toronto['price']=pd.to_numeric( df_airbnb_listing_toronto['price'], downcast="float")*0.64

# remove format errors
df_airbnb_listing_toronto['availability_365;;;']=df_airbnb_listing_toronto['availability_365;;;'].str.replace(';;;', '', regex=True)
df_airbnb_listing_toronto.rename(columns={'availability_365;;;': 'availability_365'}, inplace=True)

# remove nan in coordinate data
df_airbnb_listing_toronto.dropna(subset=['latitude', 'longitude'],inplace=True)


df_airbnb_listing_toronto.head()

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
0,1419,Beautiful home in amazing area!,1565,Alexandra,,Little Portugal,43.64617,-79.42451,Entire home/apt,300.160004,4,7,2017-12-04,0.11,1,0
1,8077,Downtown Harbourfront Private Room,22795,Kathie & Larry,,Waterfront Communities-The Island,43.64105,-79.37628,Private room,63.359997,180,169,2013-08-27,1.25,2,365
2,12604,Seaton Village Parlour Bedroom,48239,Rona,,Annex,43.66724,-79.41598,Private room,42.239998,1,0,,,1,0
3,23691,Queen Bedroom close to downtown,93825,Yohan & Sarah,,Briar Hill-Belgravia,43.69602,-79.45468,Private room,44.799999,1,217,2019-12-22,1.73,2,209
4,26654,World Class downtown @CN Tower Theatre MTCC ga...,113345,Adela,,Waterfront Communities-The Island,43.6453,-79.3894,Entire home/apt,86.400002,21,40,2020-03-20,0.34,5,216


In [248]:
# print_list_1=[]
# print_list_2=[]
# print_list_3=[]
# for d in range(0,5001, 250):
airbnb_price_private_room=np.array([])
airbnb_price_entire_home=np.array([])


# get coordinates for all neighborhoods
for lat,lon in df_to_neighborhoods[['Latitude', 'Longitude']].itertuples(index=False):

    # numpy arrays for coordinates to do matrix calculations
    lat_array=df_airbnb_listing_toronto[['latitude']].to_numpy().astype('float32')
    long_array=df_airbnb_listing_toronto[[ 'longitude']].to_numpy().astype('float32')

    lat_dist=np.absolute(lat-lat_array)*111139  # convert distance from Degrees to Meters
    long_dist=np.absolute(lon-long_array)*111139
    distance= np.sqrt( np.power(lat_dist, 2) + np.power(long_dist, 2) ) # pythagoras theorem for distance
    
    df_airbnb_listing_toronto['distance']=pd.DataFrame(distance, columns=['distance'])

    # nearby supposed to be 500 meters, but there is too many NaN values so 750 m will be used.
    df_airbnb_listing_toronto_nearby=df_airbnb_listing_toronto[df_airbnb_listing_toronto['distance'] <= 1500]
    
    private     = df_airbnb_listing_toronto_nearby[df_airbnb_listing_toronto_nearby['room_type']=='Private room']['price'].mean()
    Entire_home = df_airbnb_listing_toronto_nearby[df_airbnb_listing_toronto_nearby['room_type']=='Entire home/apt']['price'].mean()

    airbnb_price_private_room= np.append(airbnb_price_private_room, private)    
    airbnb_price_entire_home= np.append(airbnb_price_entire_home, Entire_home)
df_to_neighborhoods["airbnb_mean_price_private_room"]=airbnb_price_private_room
df_to_neighborhoods["airbnb_mean_price_entire_home"]=airbnb_price_entire_home
    # df_to_neighborhoods.head()
#     print_list_1.append(d)
#     print_list_2.append(df_to_neighborhoods['airbnb_mean_price_private_room'].isnull().sum())
#     print_list_3.append(df_to_neighborhoods['airbnb_mean_price_entire_home'].isnull().sum())

# data_collected=pd.DataFrame(data={"d": print_list_1, "room":print_list_2, "home":print_list_3})
# data_collected.head(40)

In [249]:
print(df_to_neighborhoods['airbnb_mean_price_private_room'].isnull().sum())
print(df_to_neighborhoods['airbnb_mean_price_entire_home'].isnull().sum())

0
0


### New York

In [130]:
start_time_ny = time.time()
wrong_amount_of_element_ny=[]
number_of_rows_ny=16

# AirBnb data file for 2019 #http://insideairbnb.com/get-the-data.html

count_ny=0
airbnb_calend_list_ny=[]
with open('../datafiles/airbnb_listings_sum_new_york.csv', newline='',encoding = 'cp850') as csvfile:    
    csv_reader_data_ny = csv.reader(csvfile)
    for row in csv_reader_data_ny: 
        next_row = []

        if count_ny >=1 and np.shape(row)[0]==1: # if count is not header and has "" around the all elements

            row[0]=row[0].replace("\n","")
            redone_csv_reading = csv.reader(row)
            for new_row in redone_csv_reading:

                if np.shape(new_row)[0]==number_of_rows_ny:
                    next_row = new_row

                else:
                    wrong_amount_of_element_ny.append(np.shape(new_row)[0])

        elif count_ny >=1 and np.shape(row)[0]==number_of_rows_ny:
            next_row = row

        elif count_ny==0:

            headers_row = row

        if count_ny >=1:     
            airbnb_calend_list_ny.append(next_row)

        count_ny = count_ny + 1
        if count_ny == 20000:
            break
# print(airbnb_calend_list)
# print(np.shape(airbnb_calend_list),np.shape(headers_row))
df_airbnb_listing_ny = pd.DataFrame(data=airbnb_calend_list_ny, columns= headers_row)
# print(df_airbnb_listing_ny.head())

print("Number of elements in a data row and their count: ",Counter(wrong_amount_of_element_ny))
print("--- %s seconds ---" % (time.time() - start_time_ny))

Number of elements in a data row and their count:  Counter({2: 32, 15: 28, 1: 3, 18: 1})
--- 0.29106640815734863 seconds ---


In [132]:
df_airbnb_listing_ny.head(3)

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365;;;;;
0,2595,Skylit Midtown Castle,2845,Jennifer,Manhattan,Midtown,40.75362,-73.98377,Entire home/apt,112.000000,3,48,2019-11-04,0.37,2,365;;;;;
1,3831,"Whole flr w/private bdrm, bath & kitchen(pls r...",4869,LisaRoxanne,Brooklyn,Clinton Hill,40.68514,-73.95976,Entire home/apt,48.639999,1,349,2020-08-23,4.82,1,272;;;;;
2,5121,BlissArtsSpace!,7356,Garon,Brooklyn,Bedford-Stuyvesant,40.68688,-73.95596,Private room,38.399998,29,50,2019-12-02,0.36,1,365;;;;;


In [133]:
# change currency from canadian dollar to Euro
df_airbnb_listing_ny['price']=pd.to_numeric( df_airbnb_listing_ny['price'], downcast="float")*0.64

# remove format errors
df_airbnb_listing_ny['availability_365;;;;;']=df_airbnb_listing_ny['availability_365;;;;;'].str.replace(';;;;;', '', regex=True)
df_airbnb_listing_ny.rename(columns={'availability_365;;;;;': 'availability_365'}, inplace=True)

# remove nan in coordinate data
df_airbnb_listing_ny.dropna(subset=['latitude', 'longitude'],inplace=True)


df_airbnb_listing_ny.head()

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
0,2595,Skylit Midtown Castle,2845,Jennifer,Manhattan,Midtown,40.75362,-73.98377,Entire home/apt,71.680000,3,48,2019-11-04,0.37,2,365
1,3831,"Whole flr w/private bdrm, bath & kitchen(pls r...",4869,LisaRoxanne,Brooklyn,Clinton Hill,40.68514,-73.95976,Entire home/apt,31.129599,1,349,2020-08-23,4.82,1,272
2,5121,BlissArtsSpace!,7356,Garon,Brooklyn,Bedford-Stuyvesant,40.68688,-73.95596,Private room,24.575998,29,50,2019-12-02,0.36,1,365
3,5136,"Spacious Brooklyn Duplex, Patio + Garden",7378,Rebecca,Brooklyn,Sunset Park,40.6612,-73.99423,Entire home/apt,71.680000,7,1,2014-01-02,0.01,1,352
4,5178,Large Furnished Room Near B'wayÒÇÇ,8967,Shunichi,Manhattan,Hell's Kitchen,40.76489,-73.98493,Private room,32.358398,2,473,2020-03-15,3.42,1,346


In [242]:
# print_list_1=[]
# print_list_2=[]
# print_list_3=[]
# for d in range(0,5001, 250):
airbnb_price_private_room_ny=np.array([])
airbnb_price_entire_home_ny=np.array([])


# get coordinates for all neighborhoods
for lat,lon in df_ny_neighborhoods[['Latitude', 'Longitude']].itertuples(index=False):

    # numpy arrays for coordinates to do matrix calculations
    lat_array=df_airbnb_listing_ny[['latitude']].to_numpy().astype('float32')
    long_array=df_airbnb_listing_ny[[ 'longitude']].to_numpy().astype('float32')

    lat_dist=np.absolute(lat-lat_array)*111139  # convert distance from Degrees to Meters
    long_dist=np.absolute(lon-long_array)*111139
    distance= np.sqrt( np.power(lat_dist, 2) + np.power(long_dist, 2) ) # pythagoras theorem for distance
    
    df_airbnb_listing_ny['distance']=pd.DataFrame(distance, columns=['distance'])

    # nearby supposed to be 500 meters, but there is too many NaN values so 750 m will be used.
    df_airbnb_listing_ny_nearby=df_airbnb_listing_ny[df_airbnb_listing_ny['distance'] <=1500]
    
    private     = df_airbnb_listing_ny_nearby[df_airbnb_listing_ny_nearby['room_type']=='Private room']['price'].mean()
    Entire_home = df_airbnb_listing_ny_nearby[df_airbnb_listing_ny_nearby['room_type']=='Entire home/apt']['price'].mean()

    airbnb_price_private_room_ny= np.append(airbnb_price_private_room_ny, private)    
    airbnb_price_entire_home_ny= np.append(airbnb_price_entire_home_ny, Entire_home)

df_ny_neighborhoods["airbnb_mean_price_private_room"]=airbnb_price_private_room_ny
df_ny_neighborhoods["airbnb_mean_price_entire_home"]=airbnb_price_entire_home_ny
df_ny_neighborhoods.head()

#     print_list_1.append(d)
#     print_list_2.append(df_ny_neighborhoods['airbnb_mean_price_private_room'].isnull().sum())
#     print_list_3.append(df_ny_neighborhoods['airbnb_mean_price_entire_home'].isnull().sum())

# data_collected=pd.DataFrame(data={"d": print_list_1, "room":print_list_2, "home":print_list_3})
# data_collected.head(40)

,Neighborhood,Borough,Latitude,Longitude,mean_temp_last5days,most_freq_weather_last5days,2nd_most_freq_weather_last5days,airbnb_mean_price_private_room,airbnb_mean_price_entire_home
0,Wakefield,Bronx,40.9062,-73.8555,-0.36375,Snow,Clear,NaN,47.513599
1,Co-op City,Bronx,40.8739,-73.8294,-0.00375,Snow,Clear,26.869757,62.361595
2,Eastchester,Bronx,40.9611,-73.8065,-0.45250,Snow,Clear,NaN,NaN
3,Fieldston,Bronx,40.8944,-73.9033,-0.38250,Clear,Snow,30.310400,68.110619
4,Riverdale,Bronx,40.9006,-73.9064,-0.51750,Clear,Snow,34.347881,NaN


In [243]:
print(df_ny_neighborhoods['airbnb_mean_price_private_room'].isnull().sum())
print(df_ny_neighborhoods['airbnb_mean_price_entire_home'].isnull().sum())

62
55


## Data analysis

In [306]:
df_ny_neighborhoods[['airbnb_mean_price_private_room', 'airbnb_mean_price_entire_home']].describe()

,airbnb_mean_price_private_room,airbnb_mean_price_entire_home
count,213.000000,220.000000
mean,35.371926,76.209550
std,35.953590,28.156285
min,12.287999,31.129599
25%,29.081598,64.432348
50%,32.362415,75.957401
75%,36.311817,81.919998
max,543.744019,359.475159


In [307]:
df_to_neighborhoods[['airbnb_mean_price_private_room', 'airbnb_mean_price_entire_home']].describe()

,airbnb_mean_price_private_room,airbnb_mean_price_entire_home
count,55.000000,55.000000
mean,47.199975,119.863004
std,8.794838,26.927950
min,28.031998,65.371422
25%,43.047617,106.100895
50%,46.302608,114.669151
75%,49.962629,119.575184
max,84.662849,227.199997



### Define Foursquare Credentials and Version

In [251]:
# the credentials for Foursquared is stored in another file
CLIENT_ID=config.client_id
CLIENT_SECRET=config.client_secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value
RADIUS = 1500 # define radius

# Explore all the neighborhoods

### Toronto

In [265]:
toronto_venues = func.getNearbyVenues(names=df_to_neighborhoods['Neighborhood'],
                                   latitudes=df_to_neighborhoods['Latitude'],
                                   longitudes=df_to_neighborhoods['Longitude'], CLIENT_ID=CLIENT_ID, CLIENT_SECRET=CLIENT_SECRET, VERSION='20180605', LIMIT=LIMIT,radius=RADIUS
                                  )

Parkwoods
Victoria Village
Lawrence Manor, Lawrence Heights
Islington Avenue, Humber Valley Village
Malvern, Rouge
Don Mills
Glencairn
Don Mills
Woodbine Heights
St. James Town
Humewood-Cedarvale
Guildwood, Morningside, West Hill
The Beaches
Berczy Park
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Thorncliffe Park
Dufferin, Dovercourt Village
Scarborough Village
Bayview Village
Downsview
The Danforth West, Riverdale
York Mills, Silver Hills
Downsview
Humber Summit
Willowdale, Newtonbrook
Downsview
Studio District
Bedford Park, Lawrence Manor East
Willowdale, Willowdale East
Downsview
Lawrence Park
Roselawn
Runnymede, The Junction North
Weston
York Mills West
Davisville North
Forest Hill North & West, Forest Hill Road Park
High Park, The Junction South
Westmount
Willowdale, Willowdale West
Parkdale, Roncesvalles
Agincourt
Davisville
University of Toronto, Harbord
Runnymede, Swansea
Rosedale
Alderwood, Long Branch
Northwest, West Humber - Clairville
Upper Rouge


In [266]:
print(toronto_venues.shape)
toronto_venues.head()

(3780, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.761124,-79.324059,Allwyn's Bakery,43.759840,-79.324719,Caribbean Restaurant
1,Parkwoods,43.761124,-79.324059,Tim Hortons,43.760668,-79.326368,Café
2,Parkwoods,43.761124,-79.324059,LCBO,43.757774,-79.314257,Liquor Store
3,Parkwoods,43.761124,-79.324059,A&W,43.760643,-79.326865,Fast Food Restaurant
4,Parkwoods,43.761124,-79.324059,Fat Bastard Burrito Co,43.774146,-79.322276,Burrito Place


In [267]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 289 uniques categories.


#### Analyze each Neighbourhood

In [268]:

# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Zoo,Afghan Restaurant,Airport,American Restaurant,Amphitheater,Animal Shelter,Antique Shop,Argentinian Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,Automotive Shop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Beach Bar,Beer Bar,Beer Store,Belgian Restaurant,Big Box Store,Bike Shop,Bistro,Bookstore,Botanical Garden,Boutique,Bowling Alley,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Butcher,Café,Cajun / Creole Restaurant,Campground,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Casino,Castle,Cemetery,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Rec Center,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Czech Restaurant,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Escape Room,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Gas Station,Gastropub,Gay Bar,German Restaurant,Gift Shop,Go Kart Track,Golf Course,Golf Driving Range,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hakka Restaurant,Hardware Store,Health & Beauty Service,Health Food Store,High School,Historic Site,History Museum,Hobby Shop,Hockey Arena,Hong Kong Restaurant,Hookah Bar,Hostel,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Indoor Play Area,Intersection,Italian Restaurant,Japanese Restaurant,Jewelry Store,Jewish Restaurant,Juice Bar,Kitchen Supply Store,Korean Restaurant,Lake,Latin American Restaurant,Leather Goods Store,Light Rail Station,Lighting Store,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts School,Massage Studio,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Monument / Landmark,Moroccan Restaurant,Motorcycle Shop,Movie Theater,Museum,Music School,Music Store,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Nightclub,Noodle House,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Supply Store,Paintball Field,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Pide Place,Pie Shop,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Pool Hall,Portuguese Restaurant,Print Shop,Pub,Racecourse,Racetrack,Ramen Restaurant,Record Shop,Recreation Center,Rental Car Location,Restaurant,River,Rock Climbing Spot,Rock Club,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Science Museum,Seafood Restaurant,Shoe Store,Shopping Mall,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soccer Stadium,Soup Place,South American Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Sports Club,Sri Lankan Restaurant,Stationery Store,Steakhouse,Street Art,Supermarket,Sushi Restaurant,Swiss Restaurant,Syrian Restaurant,Taco Place,Taiwanese Restaurant,Tapas Restaurant,Tattoo Parlor,Tea Room,Tennis Court,Tex-Mex Restaurant,Thai Restaurant,Theater,Theme Park,Theme Park Ride / Attraction,Theme Restaurant,Thrift / Vintage Store,Tibetan Restaurant,Toy / Game Store,Track,Trail,Train Station,Tree,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,V

In [269]:

toronto_onehot.shape

(3780, 289)

Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [270]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped.head()

,Neighborhood,Zoo,Afghan Restaurant,Airport,American Restaurant,Amphitheater,Animal Shelter,Antique Shop,Argentinian Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,Automotive Shop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Beach Bar,Beer Bar,Beer Store,Belgian Restaurant,Big Box Store,Bike Shop,Bistro,Bookstore,Botanical Garden,Boutique,Bowling Alley,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Butcher,Café,Cajun / Creole Restaurant,Campground,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Casino,Castle,Cemetery,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Rec Center,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Czech Restaurant,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Escape Room,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Gas Station,Gastropub,Gay Bar,German Restaurant,Gift Shop,Go Kart Track,Golf Course,Golf Driving Range,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hakka Restaurant,Hardware Store,Health & Beauty Service,Health Food Store,High School,Historic Site,History Museum,Hobby Shop,Hockey Arena,Hong Kong Restaurant,Hookah Bar,Hostel,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Indoor Play Area,Intersection,Italian Restaurant,Japanese Restaurant,Jewelry Store,Jewish Restaurant,Juice Bar,Kitchen Supply Store,Korean Restaurant,Lake,Latin American Restaurant,Leather Goods Store,Light Rail Station,Lighting Store,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts School,Massage Studio,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Monument / Landmark,Moroccan Restaurant,Motorcycle Shop,Movie Theater,Museum,Music School,Music Store,Music Venue,Nail Salon,New American Restaurant,Nightclub,Noodle House,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Supply Store,Paintball Field,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Pide Place,Pie Shop,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Pool Hall,Portuguese Restaurant,Print Shop,Pub,Racecourse,Racetrack,Ramen Restaurant,Record Shop,Recreation Center,Rental Car Location,Restaurant,River,Rock Climbing Spot,Rock Club,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Science Museum,Seafood Restaurant,Shoe Store,Shopping Mall,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soccer Stadium,Soup Place,South American Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Sports Club,Sri Lankan Restaurant,Stationery Store,Steakhouse,Street Art,Supermarket,Sushi Restaurant,Swiss Restaurant,Syrian Restaurant,Taco Place,Taiwanese Restaurant,Tapas Restaurant,Tattoo Parlor,Tea Room,Tennis Court,Tex-Mex Restaurant,Thai Restaurant,Theater,Theme Park,Theme Park Ride / Attraction,Theme Restaurant,Thrift / Vintage Store,Tibetan Restaurant,Toy / Game Store,Track,Trail,Train Station,Tree,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,V

In [271]:
toronto_grouped.shape

(51, 289)

Let's print each neighborhood along with the top 5 most common venues

In [272]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agincourt----
                  venue  freq
0    Chinese Restaurant  0.11
1  Cantonese Restaurant  0.04
2            Restaurant  0.04
3                  Bank  0.04
4        Sandwich Place  0.04


----Alderwood, Long Branch----
         venue  freq
0  Coffee Shop  0.06
1         Park  0.06
2   Restaurant  0.04
3          Bar  0.04
4         Café  0.04


----Bayview Village----
         venue  freq
0  Coffee Shop  0.11
1         Park  0.06
2  Pizza Place  0.05
3         Café  0.05
4     Pharmacy  0.05


----Bedford Park, Lawrence Manor East----
                venue  freq
0         Coffee Shop  0.09
1    Sushi Restaurant  0.07
2  Italian Restaurant  0.07
3              Bakery  0.06
4          Restaurant  0.04


----Berczy Park----
                venue  freq
0         Coffee Shop  0.12
1               Hotel  0.08
2                Café  0.07
3  Italian Restaurant  0.04
4          Restaurant  0.04


----Cedarbrae----
                  venue  freq
0  Fast Food Restaurant  0.12
1        

Let's put that into a pandas dataframe

First, let's write a function to sort the venues in descending order

In [273]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframe and display the top 10 venues for each neighborhood.

In [274]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Chinese Restaurant,Bank,Sandwich Place,Cantonese Restaurant,Restaurant,Fast Food Restaurant,Filipino Restaurant,Liquor Store,Shopping Mall,Supermarket
1,"Alderwood, Long Branch",Coffee Shop,Park,Pizza Place,Bar,Bank,Discount Store,Toy / Game Store,Light Rail Station,Restaurant,Department Store
2,Bayview Village,Coffee Shop,Park,Café,Pharmacy,Pizza Place,Trail,Shoe Store,Spa,Chinese Restaurant,Furniture / Home Store
3,"Bedford Park, Lawrence Manor East",Coffee Shop,Italian Restaurant,Sushi Restaurant,Bakery,Restaurant,Pub,Sandwich Place,Café,Thai Restaurant,Fast Food Restaurant
4,Berczy Park,Coffee Shop,Hotel,Café,Restaurant,Italian Restaurant,Pizza Place,Park,Plaza,Farmers Market,Theater


### New York

In [252]:
ny_venues = func.getNearbyVenues(names=df_ny_neighborhoods['Neighborhood'],
                                   latitudes=df_ny_neiny_venuesghborhoods['Latitude'],
                                   longitudes=df_ny_neighborhoods['Longitude'], CLIENT_ID=CLIENT_ID, CLIENT_SECRET=CLIENT_SECRET, VERSION='20180605', LIMIT=LIMIT,radius=RADIUS
                                  )

Wakefield
Co-op City
Eastchester
Fieldston
Riverdale
Kingsbridge
Marble Hill
Woodlawn
Norwood
Baychester
Pelham Parkway
City Island
Bedford Park
University Heights
Morris Heights
Fordham
East Tremont
West Farms
High  Bridge
Melrose
Mott Haven
Port Morris
Longwood
Hunts Point
Morrisania
Clason Point
Throgs Neck
Country Club
Parkchester
Westchester Square
Van Nest
Morris Park
Belmont
Spuyten Duyvil
Pelham Bay
Schuylerville
Edgewater Park
Castle Hill
Concourse
Unionport
Bay Ridge
Bensonhurst
Sunset Park
Greenpoint
Gravesend
Brighton Beach
Sheepshead Bay
Flatbush
Crown Heights
East Flatbush
Kensington
Windsor Terrace
Prospect Heights
Brownsville
Williamsburg
Bushwick
Bedford Stuyvesant
Brooklyn Heights
Cobble Hill
Carroll Gardens
Red Hook
Gowanus
Fort Greene
Park Slope
Cypress Hills
East New York
Starrett City
Canarsie
Flatlands
Mill Island
Manhattan Beach
Coney Island
Bath Beach
Borough Park
Dyker Heights
Gerritsen Beach
Marine Park
Clinton Hill
Sea Gate
Downtown
Boerum Hill
Ocean Hill
Ci

In [253]:
print(ny_venues.shape)
ny_venues.head()

(21250, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Wakefield,40.906197,-73.855511,CVS pharmacy,40.905088,-73.858033,Pharmacy
1,Wakefield,40.906197,-73.855511,Avitable Bros. Deli,40.902441,-73.863791,Deli / Bodega
2,Wakefield,40.906197,-73.855511,Anna Artuso's Pastry Shop,40.902991,-73.865839,Bakery
3,Wakefield,40.906197,-73.855511,The Heritage,40.903309,-73.865991,Pub
4,Wakefield,40.906197,-73.855511,New York Bagels Deli & Catering,40.914297,-73.859077,Bagel Shop


In [254]:
print('There are {} uniques categories.'.format(len(ny_venues['Venue Category'].unique())))

There are 461 uniques categories.


#### Analyze each Neighbourhood

In [256]:
# one hot encoding
ny_onehot = pd.get_dummies(ny_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
ny_onehot['Neighborhood'] = ny_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [ny_onehot.columns[-1]] + list(ny_onehot.columns[:-1])
ny_onehot = ny_onehot[fixed_columns]

ny_onehot.head()

,Zoo Exhibit,ATM,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,Airport Lounge,Airport Service,Airport Tram,American Restaurant,Animal Shelter,Antique Shop,Aquarium,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Auditorium,Australian Restaurant,Austrian Restaurant,Auto Dealership,Auto Garage,Auto Workshop,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Bath House,Beach,Beach Bar,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Border Crossing,Botanical Garden,Boutique,Bowling Alley,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Buffet,Building,Burger Joint,Burmese Restaurant,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Campground,Candy Store,Cantonese Restaurant,Car Wash,Caribbean Restaurant,Casino,Castle,Caucasian Restaurant,Cemetery,Check Cashing Service,Cheese Shop,Chinese Restaurant,Chocolate Shop,Christmas Market,Church,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,College Arts Building,College Basketball Court,College Gym,Colombian Restaurant,Comedy Club,Comfort Food Restaurant,Comic Shop,Community Center,Concert Hall,Construction & Landscaping,Convenience Store,Cooking School,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cultural Center,Cupcake Shop,Cycle Studio,Czech Restaurant,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distillery,Dive Bar,Doctor's Office,Dog Run,Donut Shop,Dosa Place,Dry Cleaner,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Elementary School,Empanada Restaurant,English Restaurant,Escape Room,Ethiopian Restaurant,Event Service,Event Space,Exhibit,Fabric Shop,Factory,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Film Studio,Fish & Chips Shop,Fish Market,Fishing Store,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Service,Food Stand,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Gluten-free Restaurant,Go Kart Track,Golf Course,Golf Driving Range,Gourmet Shop,Greek Restaurant,Grocery Store,Gun Range,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Halal Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Heliport,Herbs & Spices Store,High School,Himalayan Restaurant,Historic Site,History Museum,Hobby Shop,Home Service,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,Hunting Supply,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Indoor Play Area,Intersection,Irish Pub,Island,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Kitchen Supply Store,Kofte Place,Korean Restaurant,Kosher Restaurant,Lake,Latin American Restaurant,Laundromat,Laundry Service,Lawyer,Lebanese Restaurant,Library,Lighthouse,Lingerie Store,Liquor Store,Locksmith,Lounge,Luggage Store,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts School,Massage Studio,Mattress Store,Medical Center,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Modern Greek Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Motel,Motorcycle Shop,Mountain,Movie Theater,Moving Target,Multiplex,Museum,Music School,Music Store,Mu

In [258]:
ny_onehot.shape

(21250, 461)

Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [259]:
ny_grouped = ny_onehot.groupby('Neighborhood').mean().reset_index()
ny_grouped.head()

,Neighborhood,Zoo Exhibit,ATM,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,Airport Lounge,Airport Service,Airport Tram,American Restaurant,Animal Shelter,Antique Shop,Aquarium,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Auditorium,Australian Restaurant,Austrian Restaurant,Auto Dealership,Auto Garage,Auto Workshop,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Bath House,Beach,Beach Bar,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Border Crossing,Botanical Garden,Boutique,Bowling Alley,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Buffet,Building,Burger Joint,Burmese Restaurant,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Campground,Candy Store,Cantonese Restaurant,Car Wash,Caribbean Restaurant,Casino,Castle,Caucasian Restaurant,Cemetery,Check Cashing Service,Cheese Shop,Chinese Restaurant,Chocolate Shop,Christmas Market,Church,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,College Arts Building,College Basketball Court,College Gym,Colombian Restaurant,Comedy Club,Comfort Food Restaurant,Comic Shop,Community Center,Concert Hall,Construction & Landscaping,Convenience Store,Cooking School,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cultural Center,Cupcake Shop,Cycle Studio,Czech Restaurant,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distillery,Dive Bar,Doctor's Office,Dog Run,Donut Shop,Dosa Place,Dry Cleaner,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Elementary School,Empanada Restaurant,English Restaurant,Escape Room,Ethiopian Restaurant,Event Service,Event Space,Exhibit,Fabric Shop,Factory,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Film Studio,Fish & Chips Shop,Fish Market,Fishing Store,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Service,Food Stand,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Gluten-free Restaurant,Go Kart Track,Golf Course,Golf Driving Range,Gourmet Shop,Greek Restaurant,Grocery Store,Gun Range,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Halal Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Heliport,Herbs & Spices Store,High School,Himalayan Restaurant,Historic Site,History Museum,Hobby Shop,Home Service,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,Hunting Supply,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Indoor Play Area,Intersection,Irish Pub,Island,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Kitchen Supply Store,Kofte Place,Korean Restaurant,Kosher Restaurant,Lake,Latin American Restaurant,Laundromat,Laundry Service,Lawyer,Lebanese Restaurant,Library,Lighthouse,Lingerie Store,Liquor Store,Locksmith,Lounge,Luggage Store,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts School,Massage Studio,Mattress Store,Medical Center,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Modern Greek Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Motel,Motorcycle Shop,Mountain,Movie Theater,Moving Target,Multiplex,Museum,Music School,M

In [260]:
ny_grouped.shape

(270, 461)

Let's print each neighborhood along with the top 5 most common venues

In [261]:
num_top_venues = 5

for hood in ny_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = ny_grouped[ny_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')


2  Fast Food Restaurant  0.05
3              Pharmacy  0.04
4        Sandwich Place  0.04


----Morris Park----
                venue  freq
0         Pizza Place  0.15
1          Donut Shop  0.06
2      Sandwich Place  0.05
3  Italian Restaurant  0.05
4         Coffee Shop  0.05


----Morrisania----
            venue  freq
0   Grocery Store  0.08
1  Sandwich Place  0.07
2      Donut Shop  0.07
3        Pharmacy  0.07
4     Pizza Place  0.06


----Mott Haven----
                venue  freq
0  Mexican Restaurant  0.07
1          Donut Shop  0.05
2         Pizza Place  0.05
3               Diner  0.04
4                 Gym  0.04


----Mount Hope----
                venue  freq
0   Health Food Store   1.0
1         Zoo Exhibit   0.0
2                Park   0.0
3  Photography Studio   0.0
4            Pharmacy   0.0


----Murray Hill----
                  venue  freq
0     Korean Restaurant  0.07
1                 Hotel  0.06
2           Coffee Shop  0.05
3  Gym / Fitness Center  0.04
4   

Let's put that into a pandas dataframe

Now let's create the new dataframe and display the top 10 venues for each neighborhood. (use the function to sort the venues in descending order)

In [264]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted_ny = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted_ny['Neighborhood'] = ny_grouped['Neighborhood']

for ind in np.arange(ny_grouped.shape[0]):
    neighborhoods_venues_sorted_ny.iloc[ind, 1:] = return_most_common_venues(ny_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted_ny.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Annadale,Pizza Place,Sushi Restaurant,Pharmacy,Trail,Coffee Shop,Restaurant,Train Station,Diner,Fast Food Restaurant,Bar
1,Arlington,Pizza Place,Mexican Restaurant,American Restaurant,Diner,Fast Food Restaurant,Asian Restaurant,Hot Dog Joint,Gift Shop,Coffee Shop,Grocery Store
2,Arrochar,Italian Restaurant,Bus Stop,Bank,Deli / Bodega,Grocery Store,Bagel Shop,Liquor Store,Pizza Place,Baseball Field,Bakery
3,Arverne,Donut Shop,Surf Spot,Beach,Pizza Place,Bank,Metro Station,Harbor / Marina,Park,Gym / Fitness Center,Thai Restaurant
4,Astoria,Pizza Place,Bar,Café,Park,Greek Restaurant,Grocery Store,Bakery,Italian Restaurant,Gym,Bubble Tea Shop


### merge all neighborhoods

In [275]:
all_grouped = pd.concat([toronto_grouped, ny_grouped], sort=False)
df_all_neighborhoods = pd.concat([df_to_neighborhoods, df_ny_neighborhoods], sort=False)

In [276]:
neighborhoods_venues_sorted_all = pd.concat([neighborhoods_venues_sorted, neighborhoods_venues_sorted_ny], sort=False)

In [281]:
neighborhoods_venues_sorted_all.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Chinese Restaurant,Bank,Sandwich Place,Cantonese Restaurant,Restaurant,Fast Food Restaurant,Filipino Restaurant,Liquor Store,Shopping Mall,Supermarket
1,"Alderwood, Long Branch",Coffee Shop,Park,Pizza Place,Bar,Bank,Discount Store,Toy / Game Store,Light Rail Station,Restaurant,Department Store
2,Bayview Village,Coffee Shop,Park,Café,Pharmacy,Pizza Place,Trail,Shoe Store,Spa,Chinese Restaurant,Furniture / Home Store
3,"Bedford Park, Lawrence Manor East",Coffee Shop,Italian Restaurant,Sushi Restaurant,Bakery,Restaurant,Pub,Sandwich Place,Café,Thai Restaurant,Fast Food Restaurant
4,Berczy Park,Coffee Shop,Hotel,Café,Restaurant,Italian Restaurant,Pizza Place,Park,Plaza,Farmers Market,Theater


## Cluster Neighborhoods

Run k-means to cluster the neighborhood into 3 clusters

In [283]:
all_grouped_clustering.tail(30)

,Zoo,Afghan Restaurant,Airport,American Restaurant,Amphitheater,Animal Shelter,Antique Shop,Argentinian Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,Automotive Shop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Beach Bar,Beer Bar,Beer Store,Belgian Restaurant,Big Box Store,Bike Shop,Bistro,Bookstore,Botanical Garden,Boutique,Bowling Alley,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Butcher,Café,Cajun / Creole Restaurant,Campground,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Casino,Castle,Cemetery,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Rec Center,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Czech Restaurant,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Escape Room,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Gas Station,Gastropub,Gay Bar,German Restaurant,Gift Shop,Go Kart Track,Golf Course,Golf Driving Range,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hakka Restaurant,Hardware Store,Health & Beauty Service,Health Food Store,High School,Historic Site,History Museum,Hobby Shop,Hockey Arena,Hong Kong Restaurant,Hookah Bar,Hostel,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Indoor Play Area,Intersection,Italian Restaurant,Japanese Restaurant,Jewelry Store,Jewish Restaurant,Juice Bar,Kitchen Supply Store,Korean Restaurant,Lake,Latin American Restaurant,Leather Goods Store,Light Rail Station,Lighting Store,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts School,Massage Studio,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Monument / Landmark,Moroccan Restaurant,Motorcycle Shop,Movie Theater,Museum,Music School,Music Store,Music Venue,Nail Salon,New American Restaurant,Nightclub,Noodle House,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Supply Store,Paintball Field,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Pide Place,Pie Shop,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Pool Hall,Portuguese Restaurant,Print Shop,Pub,Racecourse,Racetrack,Ramen Restaurant,Record Shop,Recreation Center,Rental Car Location,Restaurant,River,Rock Climbing Spot,Rock Club,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Science Museum,Seafood Restaurant,Shoe Store,Shopping Mall,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soccer Stadium,Soup Place,South American Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Sports Club,Sri Lankan Restaurant,Stationery Store,Steakhouse,Street Art,Supermarket,Sushi Restaurant,Swiss Restaurant,Syrian Restaurant,Taco Place,Taiwanese Restaurant,Tapas Restaurant,Tattoo Parlor,Tea Room,Tennis Court,Tex-Mex Restaurant,Thai Restaurant,Theater,Theme Park,Theme Park Ride / Attraction,Theme Restaurant,Thrift / Vintage Store,Tibetan Restaurant,Toy / Game Store,Track,Trail,Train Station,Tree,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Volleyball Cou

In [278]:
all_grouped_clustering.describe()

,Zoo,Afghan Restaurant,Airport,American Restaurant,Amphitheater,Animal Shelter,Antique Shop,Argentinian Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,Automotive Shop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Beach Bar,Beer Bar,Beer Store,Belgian Restaurant,Big Box Store,Bike Shop,Bistro,Bookstore,Botanical Garden,Boutique,Bowling Alley,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Butcher,Café,Cajun / Creole Restaurant,Campground,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Casino,Castle,Cemetery,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Rec Center,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Czech Restaurant,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Escape Room,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Gas Station,Gastropub,Gay Bar,German Restaurant,Gift Shop,Go Kart Track,Golf Course,Golf Driving Range,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hakka Restaurant,Hardware Store,Health & Beauty Service,Health Food Store,High School,Historic Site,History Museum,Hobby Shop,Hockey Arena,Hong Kong Restaurant,Hookah Bar,Hostel,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Indoor Play Area,Intersection,Italian Restaurant,Japanese Restaurant,Jewelry Store,Jewish Restaurant,Juice Bar,Kitchen Supply Store,Korean Restaurant,Lake,Latin American Restaurant,Leather Goods Store,Light Rail Station,Lighting Store,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts School,Massage Studio,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Monument / Landmark,Moroccan Restaurant,Motorcycle Shop,Movie Theater,Museum,Music School,Music Store,Music Venue,Nail Salon,New American Restaurant,Nightclub,Noodle House,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Supply Store,Paintball Field,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Pide Place,Pie Shop,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Pool Hall,Portuguese Restaurant,Print Shop,Pub,Racecourse,Racetrack,Ramen Restaurant,Record Shop,Recreation Center,Rental Car Location,Restaurant,River,Rock Climbing Spot,Rock Club,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Science Museum,Seafood Restaurant,Shoe Store,Shopping Mall,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soccer Stadium,Soup Place,South American Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Sports Club,Sri Lankan Restaurant,Stationery Store,Steakhouse,Street Art,Supermarket,Sushi Restaurant,Swiss Restaurant,Syrian Restaurant,Taco Place,Taiwanese Restaurant,Tapas Restaurant,Tattoo Parlor,Tea Room,Tennis Court,Tex-Mex Restaurant,Thai Restaurant,Theater,Theme Park,Theme Park Ride / Attraction,Theme Restaurant,Thrift / Vintage Store,Tibetan Restaurant,Toy / Game Store,Track,Trail,Train Station,Tree,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Volleyball Cou

In [284]:
# set number of clusters
kclusters = 3

all_grouped_clustering = all_grouped.drop('Neighborhood', 1)

# replace nan with zero
all_grouped_clustering = all_grouped_clustering.fillna(0)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(all_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([1, 1, 1, 1, 1, 0, 1, 1, 1, 1])

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [285]:
# add clustering labels
neighborhoods_venues_sorted_all.insert(0, 'Cluster Labels', kmeans.labels_)

all_merged = df_all_neighborhoods

In [286]:
all_merged

,Neighborhood,Borough,Latitude,Longitude,mean_temp_last5days,most_freq_weather_last5days,2nd_most_freq_weather_last5days,airbnb_mean_price_private_room,airbnb_mean_price_entire_home
0,Parkwoods,North York,43.7611,-79.3241,-4.40875,Clouds,Clear,39.320000,153.409348
1,Victoria Village,North York,43.7327,-79.3112,-4.18250,Clouds,Clear,50.059132,97.226662
3,"Lawrence Manor, Lawrence Heights",North York,43.7165,-79.4424,-4.33625,Clear,Rain,43.688423,104.398895
5,"Islington Avenue, Humber Valley Village",Etobicoke,43.6625,-79.5312,-3.89500,Clear,Rain,35.584000,114.806160
6,"Malvern, Rouge",Scarborough,43.8092,-79.2217,-4.61750,Clouds,Rain,47.603809,104.623161
...,...,...,...,...,...,...,...,...,...
299,Kingsbridge Heights,Bronx,40.8726,-73.9027,-0.41750,Clear,Snow,33.922329,80.983765
301,Hudson Yards,Manhattan,40.7559,-74.0005,-0.07875,Clear,Snow,36.846725,93.078026
302,Hammels,Queens,40.5888,-73.8112,0.52250,Snow,Clear,29.866663,63.522137
303,Bayswater,Queens,40.6103,-73.7672,0.46250,Snow,Clear,NaN,NaN


Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [287]:
x=np.append(all_merged['Neighborhood'].unique(), neighborhoods_venues_sorted_all['Neighborhood'].unique())

print("neighborhoods_venues_sorted_all shape ", neighborhoods_venues_sorted_all['Neighborhood'].unique().shape) 
print("all_merged shape ", np.unique(all_merged['Neighborhood']).shape) 
print("appended shape ",np.unique(x).shape)


# Consider each row as indexing tuple & get linear indexing value             
# lid = np.ravel_multi_index(x.T,x.max(0)+1)
lid=x

# Get counts and unique indices
_,idx,count = np.unique(lid,return_index=True,return_counts=True)

# See which counts are exactly 1 and select the corresponding unique indices 
# and thus the correspnding rows from input as the final output
out = x[idx[count==1]]

# elements that are not not in both 'toronto_merged' and 'neighborhoods_venues_sorted' (probably only in 'toronto_merged')
out

neighborhoods_venues_sorted_all shape  (320,)
all_merged shape  (321,)
appended shape  (321,)


array(['Longwood'], dtype=object)

In [288]:
# merge manhattan_grouped with manhattan_data to add latitude/longitude for each neighborhood
all_merged = all_merged.join(neighborhoods_venues_sorted_all.set_index('Neighborhood'), on='Neighborhood')

all_merged.head() # check the last columns!

,Neighborhood,Borough,Latitude,Longitude,mean_temp_last5days,most_freq_weather_last5days,2nd_most_freq_weather_last5days,airbnb_mean_price_private_room,airbnb_mean_price_entire_home,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Parkwoods,North York,43.7611,-79.3241,-4.40875,Clouds,Clear,39.320000,153.409348,1.0,Coffee Shop,Park,Sandwich Place,Café,Chinese Restaurant,Middle Eastern Restaurant,Bank,Gas Station,Restaurant,Fast Food Restaurant
1,Victoria Village,North York,43.7327,-79.3112,-4.18250,Clouds,Clear,50.059132,97.226662,1.0,Coffee Shop,Middle Eastern Restaurant,Grocery Store,Fast Food Restaurant,Sandwich Place,Pizza Place,Gym,Shoe Store,Supermarket,Intersection
3,"Lawrence Manor, Lawrence Heights",North York,43.7165,-79.4424,-4.33625,Clear,Rain,43.688423,104.398895,1.0,Clothing Store,Coffee Shop,Restaurant,Grocery Store,Fast Food Restaurant,Pizza Place,Pharmacy,Bank,Toy / Game Store,Mexican Restaurant
5,"Islington Avenue, Humber Valley Village",Etobicoke,43.6625,-79.5312,-3.89500,Clear,Rain,35.584000,114.806160,1.0,Pharmacy,Bank,Grocery Store,Coffee Shop,Shopping Mall,Italian Restaurant,Golf Course,Spa,French Restaurant,Tapas Restaurant
6,"Malvern, Rouge",Scarborough,43.8092,-79.2217,-4.61750,Clouds,Rain,47.603809,104.623161,0.0,Grocery Store,Fast Food Restaurant,Park,Pizza Place,Pharmacy,Sandwich Place,Sports Bar,Supermarket,Skating Rink,Salon / Barbershop


In [289]:
print("toronto_merged cluster labels ",all_merged['Cluster Labels'].unique())
# toronto_merged[toronto_merged['Cluster Labels']== np.nan]
all_merged['Cluster Labels'].isna().sum() # how many nans? 

# You either create a new cluster label for the nan rows OR you drop them:
all_merged['Cluster Labels'].replace(np.nan, kclusters, inplace=True)     # replace a number =kclusters instead of nan values 
# toronto_merged['Cluster Labels'].dropna(inplace=True)                       # drop nan rows

all_merged['Cluster Labels']=all_merged['Cluster Labels'].astype(int) # change type from float to int

toronto_merged cluster labels  [ 1.  0. nan  2.]


Finally, let's visualize the resulting clusters

find torontos coordinates:

In [ ]:
#concatenate coordinate

#?

In [299]:
address = 'USA, Binghamton'

geolocator = Nominatim(user_agent="to_explorer")
location = geolocator.geocode(address)
to_latitude = location.latitude
to_longitude = location.longitude
print('The geograpical coordinate of our centroid are {}, {}.'.format(to_latitude, to_longitude))

The geograpical coordinate of our centroid are 42.096968, -75.914341.


In [301]:

# create map
map_clusters = folium.Map(location=[to_latitude, to_longitude], zoom_start=7)

# set color scheme for the clusters
x = np.arange(kclusters+1)
ys = [i + x + (i*x)**2 for i in range(kclusters+1)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(all_merged['Latitude'], all_merged['Longitude'], all_merged['Neighborhood'], all_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

# Examine clusters

### Cluster 1

In [290]:
all_merged.loc[all_merged['Cluster Labels'] == 0, all_merged.columns[[1] + list(range(5, all_merged.shape[1]))]]


,Borough,most_freq_weather_last5days,2nd_most_freq_weather_last5days,airbnb_mean_price_private_room,airbnb_mean_price_entire_home,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
6,Scarborough,Clouds,Rain,47.603809,104.623161,0,Grocery Store,Fast Food Restaurant,Park,Pizza Place,Pharmacy,Sandwich Place,Sports Bar,Supermarket,Skating Rink,Salon / Barbershop
10,North York,Clear,Rain,46.302608,105.971352,0,Coffee Shop,Pizza Place,Bank,Fast Food Restaurant,Grocery Store,Sandwich Place,Restaurant,Italian Restaurant,Caribbean Restaurant,Middle Eastern Restaurant
14,East York,Clouds,Clear,40.480000,119.118370,0,Park,Pizza Place,Skating Rink,Sandwich Place,Athletics & Sports,Coffee Shop,Soccer Stadium,Café,Brewery,Liquor Store
22,Scarborough,Clear,Rain,28.031998,174.720001,0,Fast Food Restaurant,Pizza Place,Coffee Shop,Sandwich Place,Discount Store,Diner,Department Store,Park,Toy / Game Store,Liquor Store
26,Scarborough,Clear,Rain,37.119999,188.209213,0,Fast Food Restaurant,Bank,Sandwich Place,Pharmacy,Pizza Place,Department Store,Park,Coffee Shop,Grocery Store,Indian Restaurant
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
294,Queens,Snow,Clear,17.203199,81.919998,0,Deli / Bodega,Italian Restaurant,Pharmacy,Gym,Coffee Shop,Donut Shop,Ice Cream Shop,Diner,Cosmetics Shop,Convenience Store
299,Bronx,Clear,Snow,33.922329,80.983765,0,Pizza Place,Mexican Restaurant,Donut Shop,Diner,Park,Bakery,Latin American Restaurant,Coffee Shop,Gym,Grocery Store
302,Queens,Snow,Clear,29.866663,63.522137,0,Beach,Pizza Place,Surf Spot,Bar,Bagel Shop,Beach Bar,Metro Station,Wine Bar,Coffee Shop,Deli / Bodega
303,Queens,Snow,Clear,NaN,NaN,0,Pizza Place,Chinese Restaurant,Breakfast Spot,Metro Station,Discount Store,Pier,Fast Food Restaurant,Caribbean Restaurant,Café,Bus Stop


### Cluster 2

In [291]:
all_merged.loc[all_merged['Cluster Labels'] == 1, all_merged.columns[[1] + list(range(5, all_merged.shape[1]))]]

,Borough,most_freq_weather_last5days,2nd_most_freq_weather_last5days,airbnb_mean_price_private_room,airbnb_mean_price_entire_home,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,North York,Clouds,Clear,39.320000,153.409348,1,Coffee Shop,Park,Sandwich Place,Café,Chinese Restaurant,Middle Eastern Restaurant,Bank,Gas Station,Restaurant,Fast Food Restaurant
1,North York,Clouds,Clear,50.059132,97.226662,1,Coffee Shop,Middle Eastern Restaurant,Grocery Store,Fast Food Restaurant,Sandwich Place,Pizza Place,Gym,Shoe Store,Supermarket,Intersection
3,North York,Clear,Rain,43.688423,104.398895,1,Clothing Store,Coffee Shop,Restaurant,Grocery Store,Fast Food Restaurant,Pizza Place,Pharmacy,Bank,Toy / Game Store,Mexican Restaurant
5,Etobicoke,Clear,Rain,35.584000,114.806160,1,Pharmacy,Bank,Grocery Store,Coffee Shop,Shopping Mall,Italian Restaurant,Golf Course,Spa,French Restaurant,Tapas Restaurant
7,North York,Clouds,Clear,48.319996,157.755722,1,Coffee Shop,Clothing Store,Juice Bar,Chinese Restaurant,Sandwich Place,Bakery,Bank,Gas Station,Restaurant,Middle Eastern Restaurant
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
288,Queens,Snow,Clouds,NaN,NaN,1,Food & Drink Shop,American Restaurant,Motel,Café,Greek Restaurant,Liquor Store,Bistro,Golf Course,Spa,Convenience Store
292,Staten Island,Clouds,Rain,NaN,NaN,1,Lake,Zoo,Food,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Film Studio
295,Brooklyn,Clear,Snow,31.807014,77.410599,1,Coffee Shop,Spa,Hotel,Gym / Fitness Center,Burger Joint,Memorial Site,Park,Optical Shop,Falafel Restaurant,Plaza
296,Brooklyn,Clear,Snow,33.005753,69.993294,1,Bar,Mexican Restaurant,Pizza Place,Coffee Shop,Latin American Restaurant,Deli / Bodega,Bakery,Café,Nightclub,Brewery


### Cluster 3

In [292]:
all_merged.loc[all_merged['Cluster Labels'] == 2, all_merged.columns[[1] + list(range(5, all_merged.shape[1]))]]

,Borough,most_freq_weather_last5days,2nd_most_freq_weather_last5days,airbnb_mean_price_private_room,airbnb_mean_price_entire_home,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
280,Brooklyn,Clouds,Snow,NaN,NaN,2,Farmers Market,Zoo,Fabric Shop,Falafel Restaurant,Farm,Fast Food Restaurant,Field,Filipino Restaurant,Film Studio,Fish & Chips Shop


In [297]:
all_merged.loc[all_merged['Cluster Labels'] == 3, all_merged.columns[[1] + list(range(5, all_merged.shape[1]))]]

,Borough,most_freq_weather_last5days,2nd_most_freq_weather_last5days,airbnb_mean_price_private_room,airbnb_mean_price_entire_home,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
23,Bronx,Snow,Clouds,NaN,NaN,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# THE END

In [64]:
url = 'https://www.numbeo.com/crime/rankings.jsp'
df_city_weather = pd.read_html(url)[1]   # 0 is for the 1st table in this particular page
# df_city_weather.head()
df_city_weather.drop(['Rank'], axis=1, inplace=True) # drop rank dol since only nan
df_city_weather['Rank'] = df_city_weather.index # add index as rank, since they are ordered by rank
# df_city_weather['City'].str.split(',', expand=True)[2].unique()
df_city_weather=pd.concat([df_city_weather, df_city_weather['City'].str.split(', ', expand=True)], axis=1)
# df_city_weather
# ~pd.notna(df_city_weather[2])

In [65]:
# fixa 
# print(Counter(~pd.notna(df_city_weather[2])))
# print(Counter(df_city_weather[2]))
# print(Counter(df_city_weather[1]))
# df_city_weather[pd.notna(df_city_weather[2])][1] =  df_city_weather[pd.notna(df_city_weather[2])][1].astype(str) + ",  "
# df_city_weather[~pd.notna(df_city_weather[2])][2].str.replace(None,"a")
df_city_weather.loc[pd.notna(df_city_weather[2]), 2]=", "+df_city_weather.loc[pd.notna(df_city_weather[2]), 2].astype(str)
df_city_weather[2].fillna("",inplace=True)
# df_city_weather.head(20)
# # df_city_weather[2]

df_city_weather['Country']=df_city_weather[1].str.cat(df_city_weather[2])
# # # # df_city_weather[df_city_weather[1]!=""]
# # # # df_city_weather[df_city_weather[1]==None]
# # # df_city_weather[pd.notna(df_city_weather[2])][1] 

df_city_weather.drop([1,2,"City"],axis=1, inplace=True)
df_city_weather.rename(columns={0:'City'},inplace=True)
# print(df_city_weather.head(20))


In [66]:
# print(df_city_weather[[0,1,2, 'Country']].head(20))
# print(Counter(df_city_weather['Country']))
# print(df_city_weather.head(20))

In [67]:
# A=['aa','adsadaf', 'fafwfe','de', 'Ecuador']
# eb=pd.DataFrame({'col':A})
# eb['col2'] = 'd ' + eb['col'].astype(str)
# eb

In [68]:
start_time = time.time()
wrong_amount_of_element=[]
count=0
airbnb_calend_list=[]

with open('../datafiles/airbnb_calendar_stockholm.csv', newline='',encoding = 'cp850') as csvfile:
    csv_reader_data = csv.reader(csvfile)
    for row in csv_reader_data: 
        next_row = []
        if count >=1 and np.shape(row)[0]==1: # if count is not header and has "" around the all elements
            # print(row)
            # print(np.shape(row))

            redone_csv_reading = csv.reader(row)
            for new_row in redone_csv_reading:
                # print([new_row])
                # print(np.shape(new_row))
                if np.shape(new_row)[0]==7:
                    next_row = new_row
                    # print("1")
                    # print(new_row)
                else:
                    wrong_amount_of_element.append(np.shape(new_row)[0])
                    # print("\nError: not shape of 'new_row' ==7 ")
                    # print(new_row)

            # print("\n")
        elif count >=1 and np.shape(row)[0]==7:
            next_row = row
            # print("2")
            # print([next_row])
        elif count==0:
            # print(row)
            # print(np.shape(row))
            headers_row = row

        # else:
            # print("\nError: not shape of 'row' == 1 or 7")
            # print(row)
            # next_row = np.empty((1,7,))
            # next_row[:] = np.nan
            # # next_row=np.nan
            # print("")


        if count >=1:     
            airbnb_calend_list.append(next_row)

        count = count + 1
        if count == 15000000:
            break
# print(airbnb_calend_list)
print(np.shape(airbnb_calend_list),np.shape(headers_row))
df_airbnb_calend_stockholm = pd.DataFrame(data=airbnb_calend_list, columns= headers_row)
print(df_airbnb_calend_stockholm)
# df_airbnb_calend_stockholm.to_csv('../datafiles/airbnb_calendar_stockholm_new.csv',index=False)
print("Number of elements in row and there count: ",Counter(wrong_amount_of_element))
print("--- %s seconds ---" % (time.time() - start_time))

(1048575, 7) (7,)
        listing_id        date available      price adjusted_price  \
0           565912  2020-06-27         t  $2,461.00      $2,461.00   
1           145320  2020-06-27         f  $1,424.00      $1,424.00   
2           145320  2020-06-28         f  $1,424.00      $1,424.00   
3           145320  2020-06-29         f  $1,424.00      $1,424.00   
4           145320  2020-06-30         f  $1,424.00      $1,424.00   
...            ...         ...       ...        ...            ...   
1048570   15773429  2020-11-24         t    $250.00        $250.00   
1048571   15773429  2020-11-25         t    $250.00        $250.00   
1048572   15773429  2020-11-26         t    $250.00        $250.00   
1048573   15773429  2020-11-27         t    $250.00        $250.00   
1048574   15773429  2020-11-28         t    $250.00        $250.00   

        minimum_nights maximum_nights  
0                    4             20  
1                    3           1125  
2                    

In [69]:
df_airbnb_calend_stockholm.head()

,listing_id,date,available,price,adjusted_price,minimum_nights,maximum_nights
0,565912,2020-06-27,t,"$2,461.00","$2,461.00",4,20
1,145320,2020-06-27,f,"$1,424.00","$1,424.00",3,1125
2,145320,2020-06-28,f,"$1,424.00","$1,424.00",3,1125
3,145320,2020-06-29,f,"$1,424.00","$1,424.00",3,1125
4,145320,2020-06-30,f,"$1,424.00","$1,424.00",3,1125


In [70]:
start_time = time.time()
wrong_amount_of_element=[]
number_of_rows=16

count=0
airbnb_calend_list=[]
with open('../datafiles/airbnb_listings_sum_stockholm.csv', newline='',encoding = 'cp850') as csvfile:
    csv_reader_data = csv.reader(csvfile)
    for row in csv_reader_data: 
        next_row = []

        if count >=1 and np.shape(row)[0]==1: # if count is not header and has "" around the all elements
            # print(row)
            # print(np.shape(row))
            row[0]=row[0].replace("\n","")
            redone_csv_reading = csv.reader(row)
            for new_row in redone_csv_reading:
                # print([new_row])
                # print(np.shape(new_row))
                if np.shape(new_row)[0]==number_of_rows:
                    next_row = new_row
                    # print("1")
                    # print(new_row)
                else:
                    wrong_amount_of_element.append(np.shape(new_row)[0])
                    # print("\nError: not shape of 'new_row' ==7 ")
                    # print(new_row)
                    # print(np.shape(new_row)[0])

            # print("\n")
        elif count >=1 and np.shape(row)[0]==number_of_rows:
            next_row = row
            # print("2")
            # print([next_row])
        elif count==0:
            # print(row)
            # print(np.shape(row))
            headers_row = row

        # else:
            # print("\nError: not shape of 'row' == 1 or 7")
            # print(row)
            # next_row = np.empty((1,7,))
            # next_row[:] = np.nan
            # # next_row=np.nan
            # print("")


        if count >=1:     
            airbnb_calend_list.append(next_row)

        count = count + 1
        if count == 10000:
            break
# print(airbnb_calend_list)
# print(np.shape(airbnb_calend_list),np.shape(headers_row))
df_airbnb_listing_stockholm = pd.DataFrame(data=airbnb_calend_list, columns= headers_row)
print(df_airbnb_listing_stockholm)
# df_airbnb_calend_stockholm.to_csv('../datafiles/airbnb_calendar_stockholm_new.csv',index=False)
print("Number of elements in row and there count: ",Counter(wrong_amount_of_element))
print("--- %s seconds ---" % (time.time() - start_time))

            id                                               name    host_id  \
0       145320             In the middle of it all - with a view!     703851   
1         None                                               None       None   
2       155685                         Hornstull with water view!     748592   
3       164448        Double room in central Stockholm with Wi-Fi     784312   
4       170651                 Petit Charm Rooftop next to heaven     814021   
...        ...                                                ...        ...   
7651  43936345        Sofa in living room in the shared apartment   31129430   
7652  43936494               Bo bekv├ñmt 17 Min fr├Ñn Sthlm City!  351810596   
7653  43942253      Modern apartment close to water and the city.   49991516   
7654  43942494            Gorgeous piece of heaven  in Stockholm!  137472486   
7655  43955053  Spacious studio apartment at the core of Soder...   30842844   

     host_name neighbourhood_group     

In [71]:
df_airbnb_listing_stockholm['neighbourhood'].unique()

array(['S├Âdermalms', None, 'Norrmalms', 'Kungsholmens', 'Rinkeby-Tensta',
       'Farsta', '├älvsj├Â', '├ûstermalms', 'Skarpn├ñcks',
       'Enskede-├àrsta-Vant├Ârs', 'H├ñgersten-Liljeholmens', 'Bromma',
       'Sk├ñrholmens', 'H├ñsselby-V├ñllingby', 'Sp├Ñnga-Tensta'],
      dtype=object)

In [72]:
df_airbnb_listing_stockholm.head()

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365;
0,145320,In the middle of it all - with a view!,703851,Kim,,S├Âdermalms,59.31364,18.05256,Entire home/apt,1250,3,72,2019-06-25,1.92,2,24;
1,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
2,155685,Hornstull with water view!,748592,Robert,,S├Âdermalms,59.31535,18.03277,Entire home/apt,3145,4,22,2015-12-01,0.20,1,0;
3,164448,Double room in central Stockholm with Wi-Fi,784312,Li,,S├Âdermalms,59.31268,18.06258,Private room,597,3,322,2020-01-29,2.97,2,180;
4,170651,Petit Charm Rooftop next to heaven,814021,Marie,,S├Âdermalms,59.31568,18.03138,Entire home/apt,719,4,34,2020-03-09,0.31,1,287;


In [73]:
# # Correctly spell the names of the neighborhoods
# df_airbnb_listing_stockholm['neighbourhood'].replace('S├Âdermalms','Södermalm', inplace=True)
# df_airbnb_listing_stockholm['neighbourhood'].replace('Norrmalms','Norrmalm', inplace=True)
# df_airbnb_listing_stockholm['neighbourhood'].replace('Kungsholmens','Kungsholmen', inplace=True)
# df_airbnb_listing_stockholm['neighbourhood'].replace('├älvsj├Â','Älvsjö', inplace=True)
# df_airbnb_listing_stockholm['neighbourhood'].replace('├ûstermalms','Östermalm', inplace=True)
# df_airbnb_listing_stockholm['neighbourhood'].replace('H├ñgersten-Liljeholmens','Hägersten-Liljeholmen', inplace=True)
# df_airbnb_listing_stockholm['neighbourhood'].replace('Sk├ñrholmens','Skärholmen', inplace=True)
# df_airbnb_listing_stockholm['neighbourhood'].replace('H├ñsselby-V├ñllingby','Hässelby-Vällingby', inplace=True)
# df_airbnb_listing_stockholm['neighbourhood'].replace('Sp├Ñnga-Tensta','Spånga-Tensta', inplace=True)

In [74]:
# df_test=df_airbnb_calend_stockholm[df_airbnb_calend_stockholm['neighbourhood']=='Södermalm']
# df_test

In [75]:
# df_test[df_test['name'].str.lower().str.contains("fantastic sofo")]   

It seems like the prices are in SEK - after comparing 'price' to website price for same place

In [76]:
start_time = time.time()
wrong_amount_of_element=[]
number_of_rows=6

count=0
airbnb_calend_list=[]
with open('../datafiles/airbnb_reviews_stockholm.csv', newline='',encoding = 'cp850') as csvfile:
    csv_reader_data = csv.reader(csvfile)
    for row in csv_reader_data: 
        next_row = [] 
        # if count== 177:
        #     print(row)
        #     print(row[0])
        #     print(np.shape(row))
            # redone_csv_reading = csv.reader(row)
            # for new_row in redone_csv_reading:
            #     # print([new_row])
            #     # print(np.shape(new_row))
            #     next_row = new_row

            #     print(next_row)
        if count >=1 and np.shape(row)[0]==1: # if count is not header and has "" around the all elements
            # print(row)
            # print(np.shape(row))
            # print(row[0])
            # print(np.shape(row[0]))
            row[0]=row[0].replace("\n","")
            redone_csv_reading = csv.reader(row)
            for new_row in redone_csv_reading:
                # print(new_row)
                # print(np.shape(new_row))
                if np.shape(new_row)[0]==number_of_rows:
                    next_row = new_row
                    # print("1")
                    # print(new_row)
                else:
                    wrong_amount_of_element.append(np.shape(new_row)[0])
                    # print("\nError: not shape of 'new_row' ==7 ")
                    # print(new_row)

            # print("\n")
        elif count >=1 and np.shape(row)[0]==number_of_rows:
            next_row = row
            # print("2")
            # print([next_row])
        elif count==0:
            # print(row)
            # print(np.shape(row))
            headers_row = row

        # else:
            # print("\nError: not shape of 'row' == 1 or 7")
            # print(row)
            # next_row = np.empty((1,7,))
            # next_row[:] = np.nan
            # # next_row=np.nan
            # print("")


        if count >=1:     
            airbnb_calend_list.append(next_row)


        count = count + 1
        if count == 1000000:
            break
# print(airbnb_calend_list)
print(np.shape(airbnb_calend_list),np.shape(headers_row))
df_airbnb_review_stockholm = pd.DataFrame(data=airbnb_calend_list, columns= headers_row)
print(df_airbnb_review_stockholm)
# df_airbnb_calend_stockholm.to_csv('../datafiles/airbnb_calendar_stockholm_new.csv',index=False)
print("Number of elements in row and there count: ",Counter(wrong_amount_of_element))
print("--- %s seconds ---" % (time.time() - start_time))

(182014,) (6,)
       listing_id         id        date reviewer_id reviewer_name  \
0          145320  156423122  2017-05-30   123386382         Erwin   
1          145320  170211906  2017-07-15   123091743          Anne   
2          145320  172169175  2017-07-20       78004      Patricia   
3          145320  176647581  2017-07-31   103178743     Charlotte   
4          145320  185676021  2017-08-22     4023961     Alexander   
...           ...        ...         ...         ...           ...   
182009   43720290  629441507  2020-06-14   308451513            Lu   
182010   43720290  631087136  2020-06-21    76954984        Shahin   
182011   43724610  630964346  2020-06-21    38355251          Ines   
182012   43749955  632003080  2020-06-25    92774441       Babette   
182013   43840216  630977479  2020-06-21   251908973          Dala   

                                      comments;;;;;;;;;;;  
0       Prima plek om Stockholm te bekijken. Alles is ...  
1       Cosy and clean f

In [77]:
df_airbnb_review_stockholm.head()

,listing_id,id,date,reviewer_id,reviewer_name,comments;;;;;;;;;;;
0,145320,156423122,2017-05-30,123386382,Erwin,Prima plek om Stockholm te bekijken. Alles is ...
1,145320,170211906,2017-07-15,123091743,Anne,Cosy and clean flat in quiet neighbourhood clo...
2,145320,172169175,2017-07-20,78004,Patricia,The host canceled this reservation 37 days bef...
3,145320,176647581,2017-07-31,103178743,Charlotte,Kim's place was outstanding and comfortable. W...
4,145320,185676021,2017-08-22,4023961,Alexander,great spacious apartment in a nice residential...
